Zuveir Jameer

8 April 2025

Batch 2 which contains 5 search engines.

The code below are to run the 75 queries against an Elasticsearch instance which contains TREC 2021 Clinical Trials data
The Elasticsearch instance contains ~26,000 indexed documents.

In the code below you will run searches using the following:
1. BM25 as initial search and re-ranking using semantic search on sentence embeddings of 'detailed_descrption'
2. Bi-encoder as initial search and cross-encoding reranking with Cohere PI
3. Bi-encoder as initial search and custom built cross-encoding reranking
4. Semantic search only with sentence encoder 


In [1]:
from pprint import pprint
from elasticsearch import Elasticsearch, helpers
from credentials import USERNAME, PASSWORD
import json

In [2]:
CERT_FINGERPRINT = "24a71503f93ed6e422bc8782254a826382fadd83ca09234fa7c0297173132530"

In [3]:
# Connect using API KEY
es = Elasticsearch("https://localhost:9200",api_key="eHdSWmo1VUJOYkdYV0x0X3QyTnY6cU9kQVJ1Y0VRdmFWSTc5bzB6THoxUQ==",ssl_assert_fingerprint=CERT_FINGERPRINT,)

In [4]:
# Connect using credentials and ssl fingerprint
es = Elasticsearch("https://localhost:9200",basic_auth=(USERNAME, PASSWORD),ssl_assert_fingerprint=CERT_FINGERPRINT)

In [5]:
client_info = es.info()
print('Connected to elasticsearch')
pprint(client_info.body)

Connected to elasticsearch
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'daxPe0rzR9Gx5iOkHZsZTQ',
 'name': '200fd7a2e004',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2025-02-28T10:07:26.089129809Z',
             'build_flavor': 'default',
             'build_hash': 'a091390de485bd4b127884f7e565c0cad59b10d2',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '9.12.0',
             'minimum_index_compatibility_version': '7.0.0',
             'minimum_wire_compatibility_version': '7.17.0',
             'number': '8.17.3'}}


In [19]:
es.count(index="ir_dev_sentence_index") #Number of documents in Elasticsearch

ObjectApiResponse({'count': 26162, '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}})

In [7]:
from elasticsearch import Elasticsearch, helpers
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm  # Progress bar
# from sentence_transformers import SentenceTransformer

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


def get_embedding(text):
    """
    Compute the embedding for the given text using all-mpnet-base-v2 and mean pooling.
    Returns the embedding as a list of floats.
    """
    encoded_input = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    with torch.no_grad():
        model_output = model(**encoded_input)
    embedding_temp = mean_pooling(model_output, encoded_input['attention_mask'])
    # Normalize the embedding to unit length
    embedding = F.normalize(embedding_temp, p=2, dim=1)
    # CHANGE: Convert tensor to list for further usage (e.g., storing in ES)
    return embedding[0].cpu().numpy().tolist()

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Search on sentence embeddings only 

In [ ]:
"""
Zuveir Jameer
8 April 2025
Retrieval based on sentence embedding only
"""

import json
import numpy as np
from tqdm import tqdm
import time
import pandas as pd
from datetime import datetime

#Evaluation Metric Functions
def precision_at_k(re_ranked_docs, relevant_docs, k):
    top_k_docs = re_ranked_docs[:k]
    num_relevant_in_k = sum(1 for doc in top_k_docs 
                            if doc in relevant_docs and relevant_docs[doc] > 0)
    return num_relevant_in_k / k if k else 0.0

def recall_at_k(re_ranked_docs, relevant_docs, k):
    top_k_docs = re_ranked_docs[:k]
    total_relevant = sum(1 for doc, rel in relevant_docs.items() if rel > 0)
    if total_relevant == 0:
        return 0.0
    num_relevant_in_k = sum(1 for doc in top_k_docs 
                            if doc in relevant_docs and relevant_docs[doc] > 0)
    return num_relevant_in_k / total_relevant

def f1_at_k(re_ranked_docs, relevant_docs, k):
    p = precision_at_k(re_ranked_docs, relevant_docs, k)
    r = recall_at_k(re_ranked_docs, relevant_docs, k)
    if (p + r) == 0:
        return 0.0
    return 2 * (p * r) / (p + r)

def average_precision(re_ranked_docs, relevant_docs):
    if not relevant_docs:
        return 0.0
    
    relevant_indices = [
        i for i, doc in enumerate(re_ranked_docs)
        if doc in relevant_docs and relevant_docs[doc] > 0
    ]
    if not relevant_indices:
        return 0.0
    
    ap_sum = 0.0
    for idx, rank_pos in enumerate(relevant_indices, start=1):
        prec = precision_at_k(re_ranked_docs, relevant_docs, rank_pos+1)
        ap_sum += prec
    return ap_sum / len(relevant_indices)

def ndcg_at_k(re_ranked_docs, relevant_docs, k):
    def dcg(scores):
        return sum(( (2**s - 1) / np.log2(idx + 2)) for idx, s in enumerate(scores))
    
    top_k_docs = re_ranked_docs[:k]
    actual_scores = [relevant_docs[doc] if doc in relevant_docs else 0 for doc in top_k_docs]
    actual_dcg = dcg(actual_scores)
    
    sorted_rels = sorted(relevant_docs.values(), reverse=True)[:k]
    ideal_dcg = dcg(sorted_rels)
    
    if ideal_dcg == 0:
        return 0.0
    return actual_dcg / ideal_dcg

def mrr_at_k(re_ranked_docs, relevant_docs, k):
    for i, doc_id in enumerate(re_ranked_docs[:k], start=1):
        if doc_id in relevant_docs and relevant_docs[doc_id] > 0:
            return 1.0 / i
    return 0.0


# Data Loading
def load_jsonl(filepath):
    data = {}
    with open(filepath, "r") as f:
        for line in f:
            entry = json.loads(line)
            qid = entry["query_id"]
            # If it's a query record
            if "text" in entry:
                data[qid] = entry["text"].strip()
            else:
                # It's a qrel record
                if qid not in data:
                    data[qid] = {}
                doc_id = entry["doc_id"]
                relevance = int(entry.get("relevance", 0))
                data[qid][doc_id] = relevance
    return data

queries = load_jsonl("trec_ct_2021_query.jsonl")  
qrels = load_jsonl("trec_ct_2021_qrels.jsonl")    


def semantic_search(query_text, top_k=100):
    """
    Search the entire index using a script_score
    with 'detailed_descrption_vector' and our query vector.
    """
    query_vector = get_embedding(query_text)
    vector_query_body = {
        "size": top_k,  # put size in the body to avoid deprecation warnings
        "query": {
            "script_score": {
                "query": {
                    "bool": {
                        "must": [
                            {"exists": {"field": "detailed_descrption_vector_1"}}
                        ]
                    }
                },
                "script": {
                    "source": "cosineSimilarity(params.query_vector, 'detailed_descrption_vector_1') + 1.0",
                    "params": {"query_vector": query_vector}
                }
            }
        }
    }
    
    response = es.search(index="ir_dev_sentence_index", body=vector_query_body)
    print("Time taken to retrieve (ES only): {} ms".format(response["took"]))
    return [hit["_source"]["doc_id"] for hit in response["hits"]["hits"]]

# Evaluation
searck_K=40 
K = 10               # top K for retrieval & evaluation
RELEVANCE_THRESHOLD = 2  # Consider docs with relevance >= x as relevant for evaluation

def get_relevant_docs(qrel_dict, relevance_threshold=1):
    return {doc_id: rel for doc_id, rel in qrel_dict.items() if rel >= relevance_threshold}

def get_all_docs(qrel_dict):
    """Return the entire dictionary of {doc_id: relevance}, including relevance=0."""
    return {doc_id: rel for doc_id, rel in qrel_dict.items()}

all_metrics = []

print("Semantic-Only Retrieval (sentence embeddings)...")
for qid, query_text in tqdm(queries.items(), total=len(queries)):
    if not query_text:
        continue
    
    #Track total time
    start_time = time.time()

    # Retrieve top K docs via semantic search
    retrieved_docs = semantic_search(query_text, top_k=searck_K)

    # Mark the end time for entire query processing
    total_time = time.time() - start_time
    
    # Filter qrels to get only docs with >= threshold for evaluation
    relevant_docs = get_relevant_docs(qrels.get(qid, {}), relevance_threshold=RELEVANCE_THRESHOLD)
    
    #All docs retrieved (i.e no threshold) to compute nDCG based on graded relevance 0,1,2
    ndcg_relevant_docs = get_all_docs(qrels.get(qid, {}))
    
    # Compute metrics
    prec_val = precision_at_k(retrieved_docs, relevant_docs, K)
    rec_val = recall_at_k(retrieved_docs, relevant_docs, K)
    f_val = f1_at_k(retrieved_docs, relevant_docs, K)
    ap_val = average_precision(retrieved_docs, relevant_docs)
    ndcg_val = ndcg_at_k(retrieved_docs, ndcg_relevant_docs, K)
    mrr_val = mrr_at_k(retrieved_docs, relevant_docs, K)

    metrics = {
        "query_id": qid,
        "precision": prec_val,
        "recall": rec_val,
        "f1": f_val,
        "average_precision": ap_val,
        "ndcg": ndcg_val,
        "mrr": mrr_val,
        "time_seconds": total_time
    }
    all_metrics.append(metrics)
    print(f"Query {qid} retrieved: {retrieved_docs}")
    print(f" Query {qid}: Precision@{K}: {prec_val:.4f}, Recall@{K}: {rec_val:.4f}, F1@{K}: {f_val:.4f}, AP: {ap_val:.4f}, nDCG@{K}: {ndcg_val:.4f}, mrr@{K}: {mrr_val:.4f}")
    print(f"  Total time for query: {total_time:.2f} seconds")

# Compute overall metrics
if all_metrics:
    mean_precision = np.mean([m["precision"] for m in all_metrics])
    mean_recall = np.mean([m["recall"] for m in all_metrics])
    mean_f1 = np.mean([m["f1"] for m in all_metrics])
    mean_ap = np.mean([m["average_precision"] for m in all_metrics])
    mean_ndcg = np.mean([m["ndcg"] for m in all_metrics])
    mean_mrr = np.mean([m["mrr"] for m in all_metrics])
    avg_time = np.mean([m["time_seconds"] for m in all_metrics])


    print("\n--- Overall Evaluation (Semantic-Only) ---")
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"Date/time:{current_time}")
    print(f"Relevance threshold >= {RELEVANCE_THRESHOLD}")
    print(f"Mean Precision@{K}: {mean_precision:.4f}")
    print(f"Mean Recall@{K}: {mean_recall:.4f}")
    print(f"Mean F1@{K}: {mean_f1:.4f}")
    print(f"Mean Average Precision (MAP): {mean_ap:.4f}")
    print(f"Mean nDCG@{K}: {mean_ndcg:.4f}")
    print(f"Mean MRR@{K}: {mean_mrr:.4f}")
    print(f"Average total time per query: {avg_time:.2f} seconds")
else:
    print("No queries processed.")

# export the computed results for all each query
df = pd.DataFrame(all_metrics)
df = df[[
    "query_id",
    "precision",
    "recall",
    "f1",
    "average_precision",
    "ndcg",
    "mrr",
    "time_seconds"
]]
# Export to Excel
output_file = "sentence_embedding_metrics_output.xlsx"
df.to_excel(output_file, index=False)
print(f"Metrics successfully exported to {output_file}")

# Create a DataFrame to export the computed results for all queries
data = {
    "Test": "Sentence Emdedding Only.",
    "Date/Time": [current_time],
    "Relevance Threshold": [RELEVANCE_THRESHOLD],
    f"Mean Precision@{K}": [mean_precision],
    f"Mean Recall@{K}": [mean_recall],
    f"Mean F1@{K}": [mean_f1],
    "Mean Average Precision (MAP)": [mean_ap],
    f"Mean nDCG@{K}": [mean_ndcg],
    f"Mean MRR@{K}": [mean_mrr],
    "Avg Time per Query (s)": [avg_time],
}
df = pd.DataFrame(data)

# Export to an Excel file
xlsx_filename = "evaluation_results_sentence_embeddings_only.xlsx"
df.to_excel(xlsx_filename, index=False)

print(f"\nMetrics have been exported to {xlsx_filename}")

Semantic-Only Retrieval (sentence embeddings)...


  1%|▏         | 1/75 [00:00<00:21,  3.38it/s]

Time taken to retrieve (ES only): 13 ms
Query 1 retrieved: ['NCT00003470', 'NCT01502241', 'NCT00003537', 'NCT00003621', 'NCT00003471', 'NCT00004259', 'NCT00783393', 'NCT00003775', 'NCT03906448', 'NCT00033280', 'NCT02755987', 'NCT00028795', 'NCT03633552', 'NCT00841555', 'NCT00003375', 'NCT00473408', 'NCT00002569', 'NCT00003465', 'NCT00360828', 'NCT03022578', 'NCT00004068', 'NCT00004017', 'NCT03668522', 'NCT03641443', 'NCT00362570', 'NCT00003842', 'NCT01847235', 'NCT00003176', 'NCT00717210', 'NCT00360945', 'NCT00301418', 'NCT00761280', 'NCT00400816', 'NCT00631670', 'NCT00052624', 'NCT00430911', 'NCT00007982', 'NCT00978458', 'NCT03528642', 'NCT03224104']
 Query 1: Precision@10: 0.4000, Recall@10: 0.0851, F1@10: 0.1404, AP: 0.5880, nDCG@10: 0.6502, mrr@10: 1.0000
  Total time for query: 0.30 seconds
Time taken to retrieve (ES only): 5 ms


  3%|▎         | 2/75 [00:00<00:17,  4.10it/s]

Query 2 retrieved: ['NCT03628313', 'NCT02753023', 'NCT02395107', 'NCT03332745', 'NCT01157572', 'NCT02010177', 'NCT04061213', 'NCT03863132', 'NCT02792452', 'NCT00590135', 'NCT02531581', 'NCT04140019', 'NCT01980797', 'NCT01575249', 'NCT00624884', 'NCT02316587', 'NCT04344353', 'NCT02344199', 'NCT03867708', 'NCT04144725', 'NCT01419015', 'NCT01458860', 'NCT00294775', 'NCT01845285', 'NCT02624934', 'NCT01275339', 'NCT00699452', 'NCT04444024', 'NCT02283970', 'NCT01756222', 'NCT02146755', 'NCT01636427', 'NCT02448485', 'NCT04026542', 'NCT02481258', 'NCT03049995', 'NCT02806310', 'NCT01579058', 'NCT00176410', 'NCT02241109']
 Query 2: Precision@10: 0.3000, Recall@10: 0.0217, F1@10: 0.0405, AP: 0.4247, nDCG@10: 0.5271, mrr@10: 1.0000
  Total time for query: 0.21 seconds


...

Time taken to retrieve (ES only): 14 ms
Query 73 retrieved: ['NCT01136577', 'NCT04215601', 'NCT02625688', 'NCT04099602', 'NCT04719104', 'NCT04418180', 'NCT03942757', 'NCT03055481', 'NCT02154165', 'NCT02446951', 'NCT03183986', 'NCT03569254', 'NCT03570775', 'NCT00653874', 'NCT02613676', 'NCT02774434', 'NCT01434810', 'NCT01622699', 'NCT00635375', 'NCT04365998', 'NCT02372071', 'NCT02372058', 'NCT00114543', 'NCT01203410', 'NCT04021927', 'NCT02805296', 'NCT01470820', 'NCT02156050', 'NCT00735319', 'NCT03866213', 'NCT02356978', 'NCT03599258', 'NCT01550627', 'NCT02612727', 'NCT00692224', 'NCT04585828', 'NCT04251286', 'NCT01762189', 'NCT00288600', 'NCT03130517']
 Query 73: Precision@10: 0.2000, Recall@10: 0.1000, F1@10: 0.1333, AP: 0.2855, nDCG@10: 0.3164, mrr@10: 0.1429
  Total time for query: 0.22 seconds


 99%|█████████▊| 74/75 [00:16<00:00,  4.58it/s]

Time taken to retrieve (ES only): 14 ms
Query 74 retrieved: ['NCT00306046', 'NCT00051636', 'NCT00774020', 'NCT00480662', 'NCT00747994', 'NCT04222452', 'NCT02106455', 'NCT00894075', 'NCT02802384', 'NCT02056730', 'NCT00453128', 'NCT03993821', 'NCT03859895', 'NCT00108394', 'NCT00195936', 'NCT00066183', 'NCT00417612', 'NCT02198001', 'NCT02967042', 'NCT00973336', 'NCT01875458', 'NCT00668200', 'NCT02603042', 'NCT00415584', 'NCT04184661', 'NCT00712985', 'NCT00324714', 'NCT01839383', 'NCT03716128', 'NCT00377312', 'NCT00159419', 'NCT01306656', 'NCT00572299', 'NCT01222026', 'NCT01360099', 'NCT01419028', 'NCT02235493', 'NCT00222872', 'NCT03713671', 'NCT03027349']
 Query 74: Precision@10: 0.7000, Recall@10: 0.6364, F1@10: 0.6667, AP: 0.8748, nDCG@10: 0.7885, mrr@10: 1.0000
  Total time for query: 0.21 seconds


100%|██████████| 75/75 [00:16<00:00,  4.58it/s]

Time taken to retrieve (ES only): 11 ms
Query 75 retrieved: ['NCT00443768', 'NCT00001928', 'NCT00203060', 'NCT00199394', 'NCT04380142', 'NCT00526630', 'NCT00607451', 'NCT00148512', 'NCT00472355', 'NCT00666653', 'NCT00307450', 'NCT03956979', 'NCT00199407', 'NCT03433950', 'NCT04379050', 'NCT04750226', 'NCT04472130', 'NCT00199420', 'NCT00199355', 'NCT03568968', 'NCT01003002', 'NCT00466167', 'NCT00004733', 'NCT01970813', 'NCT02424708', 'NCT04687878', 'NCT03703570', 'NCT00794313', 'NCT03011476', 'NCT00596908', 'NCT04373317', 'NCT01892176', 'NCT01016743', 'NCT01399905', 'NCT00359723', 'NCT00758368', 'NCT03665454', 'NCT02565628', 'NCT03446807', 'NCT04858074']
 Query 75: Precision@10: 0.1000, Recall@10: 0.0071, F1@10: 0.0132, AP: 0.2103, nDCG@10: 0.3576, mrr@10: 0.5000
  Total time for query: 0.23 seconds

--- Overall Evaluation (Semantic-Only) ---
Date/time:2025-04-10 08:19:54
Relevance threshold >= 2
Mean Precision@10: 0.3160
Mean Recall@10: 0.0717
Mean F1@10: 0.1035
Mean Average Precision (

### Bi-encoder and Cross-encoder

In [ ]:
"""
Zuveir Jameer
8 Apirl 2025
Bi-encoder and cross-encoder for retrieval
"""

import json
import numpy as np
from elasticsearch import Elasticsearch
from tqdm import tqdm
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer
import time
from datetime import datetime
import pandas as pd


# Setup: Load Models for Cross-encoder for re-ranking
cross_tokenizer = AutoTokenizer.from_pretrained("cross-encoder/ms-marco-MiniLM-L6-v2")
cross_model = AutoModelForSequenceClassification.from_pretrained("cross-encoder/ms-marco-MiniLM-L6-v2")

cross_model.eval()
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
cross_model.to(device)

#Evaluation Metric Functions
def precision_at_k(re_ranked_docs, relevant_docs, k):
    top_k_docs = re_ranked_docs[:k]
    num_relevant_in_k = sum(1 for doc in top_k_docs 
                            if doc in relevant_docs and relevant_docs[doc] > 0)
    return num_relevant_in_k / k if k else 0.0

def recall_at_k(re_ranked_docs, relevant_docs, k):
    top_k_docs = re_ranked_docs[:k]
    total_relevant = sum(1 for doc, rel in relevant_docs.items() if rel > 0)
    if total_relevant == 0:
        return 0.0
    num_relevant_in_k = sum(1 for doc in top_k_docs 
                            if doc in relevant_docs and relevant_docs[doc] > 0)
    return num_relevant_in_k / total_relevant

def f1_at_k(re_ranked_docs, relevant_docs, k):
    p = precision_at_k(re_ranked_docs, relevant_docs, k)
    r = recall_at_k(re_ranked_docs, relevant_docs, k)
    if (p + r) == 0:
        return 0.0
    return 2 * (p * r) / (p + r)

def average_precision(re_ranked_docs, relevant_docs):
    if not relevant_docs:
        return 0.0
    
    relevant_indices = [
        i for i, doc in enumerate(re_ranked_docs)
        if doc in relevant_docs and relevant_docs[doc] > 0
    ]
    if not relevant_indices:
        return 0.0
    
    ap_sum = 0.0
    for idx, rank_pos in enumerate(relevant_indices, start=1):
        prec = precision_at_k(re_ranked_docs, relevant_docs, rank_pos+1)
        ap_sum += prec
    return ap_sum / len(relevant_indices)

def ndcg_at_k(re_ranked_docs, relevant_docs, k):
    def dcg(scores):
        return sum(( (2**s - 1) / np.log2(idx + 2)) for idx, s in enumerate(scores))
    
    top_k_docs = re_ranked_docs[:k]
    actual_scores = [relevant_docs[doc] if doc in relevant_docs else 0 for doc in top_k_docs]
    actual_dcg = dcg(actual_scores)
    
    sorted_rels = sorted(relevant_docs.values(), reverse=True)[:k]
    ideal_dcg = dcg(sorted_rels)
    
    if ideal_dcg == 0:
        return 0.0
    return actual_dcg / ideal_dcg

def mrr_at_k(re_ranked_docs, relevant_docs, k):
    for i, doc_id in enumerate(re_ranked_docs[:k], start=1):
        if doc_id in relevant_docs and relevant_docs[doc_id] > 0:
            return 1.0 / i
    return 0.0


# Data Loading
def load_jsonl(filename):
    with open(filename, "r") as f:
        return [json.loads(line) for line in f]

queries = load_jsonl("trec_ct_2021_query.jsonl")
qrels_list = load_jsonl("trec_ct_2021_qrels.jsonl")

# Create a dictionary mapping query_id to {doc_id: relevance}
qrels = {}
for q in qrels_list:
    qid = q["query_id"]
    if qid not in qrels:
        qrels[qid] = {}
    relevance = int(q.get("relevance", 0))
    qrels[qid][q["doc_id"]] = relevance

print(f"Loaded {len(queries)} queries")
print(f"Loaded qrels for {len(qrels)} queries")

RELEVANCE_THRESHOLD = 2

filtered_qrels = {
    qid: {doc: rel for doc, rel in rels.items() if rel >= RELEVANCE_THRESHOLD}
    for qid, rels in qrels.items()
}

ndcg_qrels = {
    qid: {doc: rel for doc, rel in rels.items()}
    for qid, rels in qrels.items()
}

# Dense Retrieval (Stage 1) Functions using Bi-Encoder
def search_detailed_descrption(query_vector, K=100):
    """
    Retrieve candidate documents using dense vector search with cosine similarity.
    Assumes that each document in ES has a pre-computed vector in the field 
    'detailed_descrption_vector_1'.
    """
    search_body = {
      "query": {
        "script_score": {
          "query": {
            "bool": {
              "must": [
                {"match_all": {}},
                {"exists": {"field": "detailed_descrption_vector_1"}}
              ]
            }
          },
          "script": {
            "source": "cosineSimilarity(params.query_vector, 'detailed_descrption_vector_1') + 1.0",
            "params": {"query_vector": query_vector}
          }
        }
      }
    }
    response = es.search(index="ir_dev_sentence_index", body=search_body, size=K)
    return [hit["_source"]["doc_id"] for hit in response['hits']['hits']]

def get_document_text(doc_id):
    """
    Retrieve the detailed description for a document given its ID.
    Make sure that the field name matches your index mapping.
    """
    query = {
        "query": {
            "term": {"doc_id.keyword": doc_id}
        }
    }
    response = es.search(index="ir_dev_sentence_index", body=query, size=1)
    hits = response.get("hits", {}).get("hits", [])
    if hits:
        doc_text = hits[0]["_source"].get("detailed_descrption", "").strip()
        return doc_text if doc_text else None
    else:
        return None

K = 20  # Number of candidates from dense retrieval
top_k=10 #Number of results returned from re-ranker

def cross_encoder_rerank(query_text, candidate_doc_ids, top_k=top_k):
    candidate_pairs = []
    valid_doc_ids = []
    for doc_id in candidate_doc_ids:
        try:
            doc_text = get_document_text(doc_id)
            if doc_text is None or not doc_text.strip():
                continue
            candidate_pairs.append((query_text, doc_text))
            valid_doc_ids.append(doc_id)
        except Exception as e:
            print(f"Error fetching doc {doc_id}: {e}")
            continue

    if not candidate_pairs:
        print("No valid candidate pairs found; returning original candidates.")
        return candidate_doc_ids

    inputs = cross_tokenizer.batch_encode_plus(
        candidate_pairs, 
        padding=True, 
        truncation=True, 
        return_tensors="pt"
    )
    inputs = {k: v.to(cross_model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = cross_model(**inputs)

    if outputs.logits.shape[-1] == 1:
        scores = outputs.logits.squeeze(-1)
    else:
        scores = torch.softmax(outputs.logits, dim=1)[:, 1]

    ranked = sorted(
        zip(valid_doc_ids, scores.tolist()), 
        key=lambda x: x[1], 
        reverse=True
    )

    # If top_k is specified, slice the top_k
    if top_k is not None:
        ranked = ranked[:top_k]

    # Return just the document IDs, in ranked order
    return [doc_id for doc_id, score in ranked]

all_metrics = []

print("Processing queries and retrieving documents...")
for query in tqdm(queries, total=len(queries)):
    qid = query["query_id"]
    query_text = query["text"]
    if not query_text.strip():
        print(f"Query {qid} is empty, skipping.")
        continue
    
    start_time = time.time()
    
    # Stage 1: Get the dense vector for the query and retrieve candidates
    query_vector = get_embedding(query_text)
    try:
        retrieved_docs = search_detailed_descrption(query_vector, K)
        print(f"Query {qid} Stage 1 retrieved docs: {retrieved_docs}")
    except Exception as e:
        print(f"Error retrieving documents for query {qid}: {e}")
        continue

    # Stage 2: Re-rank candidates using the cross-encoder
    try:
        re_ranked_docs = cross_encoder_rerank(query_text, retrieved_docs)
    except Exception as e:
        print(f"Error in cross encoder re-ranking for query {qid}: {e}")
        re_ranked_docs = retrieved_docs

    total_time = time.time() - start_time
    
    # Evaluate using filtered qrels (ensure your evaluation functions are correct)
    relevant = filtered_qrels.get(qid, {})
    ndcg_relevant = ndcg_qrels.get(qid, {})
    p10 = precision_at_k(re_ranked_docs, relevant, top_k)
    r10 = recall_at_k(re_ranked_docs, relevant, top_k)
    f1 = f1_at_k(re_ranked_docs, relevant, top_k)
    ap = average_precision(re_ranked_docs, relevant)
    ndcg = ndcg_at_k(re_ranked_docs, ndcg_relevant, top_k)
    mrr_val = mrr_at_k(re_ranked_docs, relevant, top_k)

    all_metrics.append({
        "query_id": qid, 
        "precision": p10, 
        "recall": r10, 
        "f1": f1, 
        "ap": ap, 
        "ndcg": ndcg,
        "mrr": mrr_val,
        "time_seconds": total_time
    })
    print(f"  Query {qid} stage 2 retrieved (after cross-encoder re-ranking): {re_ranked_docs}")
    print(f"   Query {qid} metrics: Precision@{top_k}: {p10:.4f}, Recall@{top_k}: {r10:.4f}, F1@{top_k}: {f1:.4f}, AP: {ap:.4f}, nDCG@{top_k}: {ndcg:.4f}, MRR@{top_k}={mrr_val:.4f}")
    print(f"    Total time for query: {total_time:.2f} seconds")
    
if all_metrics:
    avg_precision = np.mean([m["precision"] for m in all_metrics])
    avg_recall = np.mean([m["recall"] for m in all_metrics])
    avg_f1 = np.mean([m["f1"] for m in all_metrics])
    avg_ap = np.mean([m["ap"] for m in all_metrics])
    avg_ndcg = np.mean([m["ndcg"] for m in all_metrics])
    avg_mrr = np.mean([m["mrr"] for m in all_metrics])
    avg_time = np.mean([m["time_seconds"] for m in all_metrics])

    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"Date/time:{current_time}")
    print("\nBi-encoder and cross-encoder - Overall evaluation metrics:")
    print(f"Relevance Threshold: {RELEVANCE_THRESHOLD}")
    print("Mean Precision@{}: {:.4f}".format(top_k, avg_precision))
    print("Mean Recall@{}: {:.4f}".format(top_k, avg_recall))
    print("Mean F1@{}: {:.4f}".format(top_k, avg_f1))
    print("Mean Average Precision (MAP): {:.4f}".format(avg_ap))
    print("Mean nDCG@{}: {:.4f}".format(top_k, avg_ndcg))
    print(f"Mean MRR{top_k}: {avg_mrr:.4f}")
    print(f"Average total time per query: {avg_time:.2f} seconds")
else:
    print("No query metrics to average.")

# export the computed results for all each query
df = pd.DataFrame(all_metrics)
df = df[[
    "query_id",
    "precision",
    "recall",
    "f1",
    "ap",
    "ndcg",
    "mrr",
    "time_seconds"
]]
# Export to Excel
output_file = "sentence_embedding_bi_and_Cross-encoding_metrics_output.xlsx"
df.to_excel(output_file, index=False)
print(f"Metrics successfully exported to {output_file}")

# Export final computed results
data = {
    "Test": "Bi-encoder + Cross-encoder",
    "Date/Time": [current_time],
    "Relevance Threshold": [RELEVANCE_THRESHOLD],
    f"Mean Precision@{top_k}": [avg_precision],
    f"Mean Recall@{top_k}": [avg_recall],
    f"Mean F1@{top_k}": [avg_f1],
    "Mean Average Precision (MAP)": [avg_ap],
    f"Mean nDCG@{top_k}": [avg_ndcg],
    f"Mean MRR@{top_k}": [avg_mrr],
    "Avg Time per Query (s)": [avg_time],
}
df = pd.DataFrame(data)

# Export to an Excel file
xlsx_filename = "evaluation_results_sentence_embeddings_bi_and_Cross-encoding.xlsx"
df.to_excel(xlsx_filename, index=False)

print(f"\nMetrics have been exported to {xlsx_filename}")

Loaded 75 queries
Loaded qrels for 75 queries
Processing queries and retrieving documents...


  0%|          | 0/75 [00:00<?, ?it/s]/var/folders/b3/59dkzgnj0zxdhdx31d2ztm000000gn/T/ipykernel_54942/392687460.py:146: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.search(index="ir_dev_sentence_index", body=search_body, size=K)
/var/folders/b3/59dkzgnj0zxdhdx31d2ztm000000gn/T/ipykernel_54942/392687460.py:159: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.search(index="ir_dev_sentence_index", body=query, size=1)


Query 1 Stage 1 retrieved docs: ['NCT00003470', 'NCT01502241', 'NCT00003537', 'NCT00003621', 'NCT00003471', 'NCT00004259', 'NCT00783393', 'NCT00003775', 'NCT03906448', 'NCT00033280', 'NCT02755987', 'NCT00028795', 'NCT03633552', 'NCT00841555', 'NCT00003375', 'NCT00473408', 'NCT00002569', 'NCT00003465', 'NCT00360828', 'NCT03022578']


  1%|▏         | 1/75 [00:01<02:20,  1.90s/it]

  Query 1 stage 2 retrieved (after cross-encoder re-ranking): ['NCT00473408', 'NCT01502241', 'NCT00003470', 'NCT00003537', 'NCT03022578', 'NCT00003375', 'NCT03633552', 'NCT00003471', 'NCT00002569', 'NCT00003465']
   Query 1 metrics: Precision@10: 0.3000, Recall@10: 0.0638, F1@10: 0.1053, AP: 0.4028, nDCG@10: 0.5162, MRR@10=0.3333
    Total time for query: 1.90 seconds
Query 2 Stage 1 retrieved docs: ['NCT03628313', 'NCT02753023', 'NCT02395107', 'NCT03332745', 'NCT01157572', 'NCT02010177', 'NCT04061213', 'NCT03863132', 'NCT02792452', 'NCT00590135', 'NCT02531581', 'NCT04140019', 'NCT01980797', 'NCT01575249', 'NCT00624884', 'NCT02316587', 'NCT04344353', 'NCT02344199', 'NCT03867708', 'NCT04144725']


...

  Query 73 stage 2 retrieved (after cross-encoder re-ranking): ['NCT04215601', 'NCT02774434', 'NCT02446951', 'NCT01434810', 'NCT01622699', 'NCT03942757', 'NCT04099602', 'NCT02625688', 'NCT01136577', 'NCT03183986']
   Query 73 metrics: Precision@10: 0.4000, Recall@10: 0.2000, F1@10: 0.2667, AP: 0.4500, nDCG@10: 0.5321, MRR@10=0.5000
    Total time for query: 0.87 seconds
Query 74 Stage 1 retrieved docs: ['NCT00306046', 'NCT00051636', 'NCT00774020', 'NCT00480662', 'NCT00747994', 'NCT04222452', 'NCT02106455', 'NCT00894075', 'NCT02802384', 'NCT02056730', 'NCT00453128', 'NCT03993821', 'NCT03859895', 'NCT00108394', 'NCT00195936', 'NCT00066183', 'NCT00417612', 'NCT02198001', 'NCT02967042', 'NCT00973336']


 99%|█████████▊| 74/75 [01:09<00:00,  1.13it/s]

  Query 74 stage 2 retrieved (after cross-encoder re-ranking): ['NCT00894075', 'NCT00453128', 'NCT03993821', 'NCT00306046', 'NCT00417612', 'NCT00108394', 'NCT02106455', 'NCT04222452', 'NCT00747994', 'NCT02802384']
   Query 74 metrics: Precision@10: 0.4000, Recall@10: 0.3636, F1@10: 0.3810, AP: 0.3173, nDCG@10: 0.2980, MRR@10=0.2500
    Total time for query: 0.87 seconds
Query 75 Stage 1 retrieved docs: ['NCT00443768', 'NCT00001928', 'NCT00203060', 'NCT00199394', 'NCT04380142', 'NCT00526630', 'NCT00607451', 'NCT00148512', 'NCT00472355', 'NCT00666653', 'NCT00307450', 'NCT03956979', 'NCT00199407', 'NCT03433950', 'NCT04379050', 'NCT04750226', 'NCT04472130', 'NCT00199420', 'NCT00199355', 'NCT03568968']


100%|██████████| 75/75 [01:10<00:00,  1.07it/s]

  Query 75 stage 2 retrieved (after cross-encoder re-ranking): ['NCT00001928', 'NCT03568968', 'NCT00443768', 'NCT00607451', 'NCT00199394', 'NCT00307450', 'NCT00472355', 'NCT04472130', 'NCT03433950', 'NCT00199420']
   Query 75 metrics: Precision@10: 0.3000, Recall@10: 0.0213, F1@10: 0.0397, AP: 0.7917, nDCG@10: 0.5685, MRR@10=1.0000
    Total time for query: 0.88 seconds
Date/time:2025-04-10 08:22:27

Bi-encoder and cross-encoder - Overall evaluation metrics:
Relevance Threshold: 2
Mean Precision@10: 0.2813
Mean Recall@10: 0.0555
Mean F1@10: 0.0830
Mean Average Precision (MAP): 0.4291
Mean nDCG@10: 0.3792
Mean MRR10: 0.4870
Average total time per query: 0.94 seconds


### Bi-encoder and cross-encoder (with Cohere)

In [ ]:
"""
Zuveir Jameer
8 April 2025
Retrieval based on sentence embedding (bi-encoder) and cohere cross-encoder (re-ranker)
"""
import json
import numpy as np
from elasticsearch import Elasticsearch
from tqdm import tqdm
import torch
from datetime import datetime
import pandas as pd
import time

# Cohere library
import cohere

# Initialize Cohere client NOte: this has been disabled because of cost. Please create your own Cohere account with the payment details.
co = cohere.Client("AWtc72wWmPFFgrQ4bUbXqhXmqmNd4tYJr0XLZVlf")

# Data Loading
def load_jsonl(filename):
    with open(filename, "r") as f:
        return [json.loads(line) for line in f]

queries = load_jsonl("trec_ct_2021_query.jsonl")
qrels_list = load_jsonl("trec_ct_2021_qrels.jsonl")

# Create a dictionary mapping query_id to {doc_id: relevance}
qrels = {}
for q in qrels_list:
    qid = q["query_id"]
    if qid not in qrels:
        qrels[qid] = {}
    relevance = int(q.get("relevance", 0))
    qrels[qid][q["doc_id"]] = relevance

print(f"Loaded {len(queries)} queries")
print(f"Loaded qrels for {len(qrels)} queries")

#filter qrels by a relevance threshold
RELEVANCE_THRESHOLD = 2
filtered_qrels = {
    qid: {doc: rel for doc, rel in rels.items() if rel >= RELEVANCE_THRESHOLD}
    for qid, rels in qrels.items()
}

# No relevance threshold for ndcg
ndcg_qrels = {
    qid: {doc: rel for doc, rel in rels.items()}
    for qid, rels in qrels.items()
}

#Evaluation Metric Functions
def precision_at_k(re_ranked_docs, relevant_docs, k):
    top_k_docs = re_ranked_docs[:k]
    num_relevant_in_k = sum(1 for doc in top_k_docs 
                            if doc in relevant_docs and relevant_docs[doc] > 0)
    return num_relevant_in_k / k if k else 0.0

def recall_at_k(re_ranked_docs, relevant_docs, k):
    top_k_docs = re_ranked_docs[:k]
    total_relevant = sum(1 for doc, rel in relevant_docs.items() if rel > 0)
    if total_relevant == 0:
        return 0.0
    num_relevant_in_k = sum(1 for doc in top_k_docs 
                            if doc in relevant_docs and relevant_docs[doc] > 0)
    return num_relevant_in_k / total_relevant

def f1_at_k(re_ranked_docs, relevant_docs, k):
    p = precision_at_k(re_ranked_docs, relevant_docs, k)
    r = recall_at_k(re_ranked_docs, relevant_docs, k)
    if (p + r) == 0:
        return 0.0
    return 2 * (p * r) / (p + r)

def average_precision(re_ranked_docs, relevant_docs):
    if not relevant_docs:
        return 0.0
    
    relevant_indices = [
        i for i, doc in enumerate(re_ranked_docs)
        if doc in relevant_docs and relevant_docs[doc] > 0
    ]
    if not relevant_indices:
        return 0.0
    
    ap_sum = 0.0
    for idx, rank_pos in enumerate(relevant_indices, start=1):
        prec = precision_at_k(re_ranked_docs, relevant_docs, rank_pos+1)
        ap_sum += prec
    return ap_sum / len(relevant_indices)

def ndcg_at_k(re_ranked_docs, relevant_docs, k):
    def dcg(scores):
        return sum(( (2**s - 1) / np.log2(idx + 2)) for idx, s in enumerate(scores))
    
    top_k_docs = re_ranked_docs[:k]
    actual_scores = [relevant_docs[doc] if doc in relevant_docs else 0 for doc in top_k_docs]
    actual_dcg = dcg(actual_scores)
    
    sorted_rels = sorted(relevant_docs.values(), reverse=True)[:k]
    ideal_dcg = dcg(sorted_rels)
    
    if ideal_dcg == 0:
        return 0.0
    return actual_dcg / ideal_dcg

def mrr_at_k(re_ranked_docs, relevant_docs, k):
    for i, doc_id in enumerate(re_ranked_docs[:k], start=1):
        if doc_id in relevant_docs and relevant_docs[doc_id] > 0:
            return 1.0 / i
    return 0.0

def search_detailed_descrption(query_vector, K=100):
    """
    Retrieve candidate documents using dense vector search with cosine similarity.
    Each document in ES has a pre-computed vector 
    in the field 'detailed_descrption_vector_1'.
    """
    search_body = {
      "query": {
        "script_score": {
          "query": {
            "bool": {
              "must": [
                {"match_all": {}},
                {"exists": {"field": "detailed_descrption_vector_1"}}
              ]
            }
          },
          "script": {
            "source": "cosineSimilarity(params.query_vector, 'detailed_descrption_vector_1') + 1.0",
            "params": {"query_vector": query_vector}
          }
        }
      }
    }
    response = es.search(index="ir_dev_sentence_index", body=search_body, size=K)
    return [hit["_source"]["doc_id"] for hit in response['hits']['hits']]

def get_document_text(doc_id):
    """
    Retrieve the detailed description for a document given its ID.
    Make sure the field name matches your index mapping.
    """
    query = {
        "query": {
            "term": {"doc_id.keyword": doc_id}
        }
    }
    response = es.search(index="ir_dev_sentence_index", body=query, size=1)
    hits = response.get("hits", {}).get("hits", [])
    if hits:
        doc_text = hits[0]["_source"].get("detailed_descrption", "").strip()
        return doc_text if doc_text else None
    else:
        return None


# Cohere Re-Ranking (Stage 2)
def cohere_rerank(query_text, candidate_doc_ids, top_k=None):
    """
    Re-rank candidate documents using Cohere's Rerank API.
    """
    docs = []
    valid_doc_ids = []

    # Collect text for each candidate doc
    for doc_id in candidate_doc_ids:
        doc_text = get_document_text(doc_id)
        if doc_text and doc_text.strip():
            docs.append(doc_text)
            valid_doc_ids.append(doc_id)

    # If no valid docs, just return the original list
    if not docs:
        return candidate_doc_ids

    # Call Cohere's rerank endpoint
    response = co.rerank(
        query=query_text,
        documents=docs,
        top_n=len(docs)
    )
    
    # response is a RerankResponse object containing a .results list
    results = response.results  # This is a list of Reranking objects

    #To limit the final results to top_k, slice the results list
    reranked = results[:top_k] if top_k else results

    # Each item in reranked has .index, .relevance_score, .document
    # Map each item's index back to your valid_doc_ids
    reranked_ids = [valid_doc_ids[item.index] for item in reranked]
    return reranked_ids


# Main Evaluation Loop

K = 20             # Number of candidates from dense retrieval
rerank_top_k = 10  # Number of top candidates to return after re-ranking
query_metrics = []

print("Processing queries and retrieving documents...")
for query in tqdm(queries, total=len(queries)):
    qid = query["query_id"]
    query_text = query["text"]
    if not query_text.strip():
        print(f"Query {qid} is empty, skipping.")
        continue

    #Track total time
    start_time = time.time()

    # Stage 1: Dense Retrieval with Elasticsearch
    query_vector = get_embedding(query_text) 
    try:
        retrieved_docs = search_detailed_descrption(query_vector, K)
        print(f"Query {qid} Stage 1 retrieved docs: {retrieved_docs}")
    except Exception as e:
        print(f"Error retrieving docs for query {qid}: {e}")
        continue

    # Stage 2: Re-rank with Cohere
    try:
        re_ranked_docs = cohere_rerank(query_text, retrieved_docs, top_k=rerank_top_k)
    except Exception as e:
        print(f"Error in Cohere re-ranking for query {qid}: {e}")
        re_ranked_docs = retrieved_docs

    # Mark the end time for entire query processing
    total_time = time.time() - start_time

    # Evaluate with qrels
    relevant = filtered_qrels.get(qid, {})
    ndcg_relevant = ndcg_qrels.get(qid, {})
    p_val = precision_at_k(re_ranked_docs, relevant, rerank_top_k)
    r_val = recall_at_k(re_ranked_docs, relevant, rerank_top_k)
    f_val = f1_at_k(re_ranked_docs, relevant, rerank_top_k)
    ap_val = average_precision(re_ranked_docs, relevant)
    ndcg_val = ndcg_at_k(re_ranked_docs, ndcg_relevant, rerank_top_k)
    mrr_val = mrr_at_k(re_ranked_docs, relevant, rerank_top_k)
    

    query_metrics.append({
        "query_id": qid,
        "precision": p_val,
        "recall": r_val,
        "f1": f_val,
        "ap": ap_val,
        "ndcg": ndcg_val,
        "mrr": mrr_val,
        "time_seconds": total_time
    })

    # all_metrics.append(metrics)
    print(f"  Query {qid} stage 2 retrieved (after cross-encoder re-ranking): {re_ranked_docs}")
    print(f"   Query {qid}: Precision@{rerank_top_k}: {p_val:.4f}, Recall@{rerank_top_k}: {r_val:.4f}, F1@{rerank_top_k}: {f_val:.4f}, AP: {ap_val:.4f}, nDCG@{rerank_top_k}: {ndcg_val:.4f}, mrr@{rerank_top_k}: {mrr_val:.4f}")
    print(f"    Total time for query: {total_time:.2f} seconds")


# Aggregate and print results
if query_metrics:
    avg_precision = np.mean([m["precision"] for m in query_metrics])
    avg_recall    = np.mean([m["recall"] for m in query_metrics])
    avg_f1        = np.mean([m["f1"] for m in query_metrics])
    avg_ap        = np.mean([m["ap"] for m in query_metrics])
    avg_ndcg      = np.mean([m["ndcg"] for m in query_metrics])
    avg_mrr       = np.mean([m["mrr"] for m in query_metrics])
    avg_time      = np.mean([m["time_seconds"] for m in query_metrics])

    print("\nOverall evaluation metrics:")
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"Date/time:{current_time}")
    print(f"Relevance Threshold: {RELEVANCE_THRESHOLD}")
    print(f"Mean Precision@{rerank_top_k}: {avg_precision:.4f}")
    print(f"Mean Recall@{rerank_top_k}: {avg_recall:.4f}")
    print(f"Mean F1@{rerank_top_k}: {avg_f1:.4f}")
    print(f"Mean Average Precision (MAP): {avg_ap:.4f}")
    print(f"Mean nDCG@{rerank_top_k}: {avg_ndcg:.4f}")
    print(f"Mean MRR@{rerank_top_k}: {avg_mrr:.4f}")
    print(f"Average total time per query: {avg_time:.2f} seconds")
    
else:
    print("No query metrics to average.")

# export the computed results for all each query
df = pd.DataFrame(query_metrics)
df = df[[
    "query_id",
    "precision",
    "recall",
    "f1",
    "ap",
    "ndcg",
    "mrr",
    "time_seconds"
]]
# Export to Excel
output_file = "sentence_embedding_bi_and_cohere_Cross-encoding_metrics_output.xlsx"
df.to_excel(output_file, index=False)
print(f"Metrics successfully exported to {output_file}")

# Export final computed results
data = {
    "Test": "Bi-encoder + COHERE cross-encoder",
    "Date/Time": [current_time],
    "Relevance Threshold": [RELEVANCE_THRESHOLD],
    f"Mean Precision@{rerank_top_k}": [avg_precision],
    f"Mean Recall@{rerank_top_k}": [avg_recall],
    f"Mean F1@{rerank_top_k}": [avg_f1],
    "Mean Average Precision (MAP)": [avg_ap],
    f"Mean nDCG@{rerank_top_k}": [avg_ndcg],
    f"Mean MRR@{rerank_top_k}": [avg_mrr],
    "Avg Time per Query (s)": [avg_time],
}
df = pd.DataFrame(data)

# Export to an Excel file
xlsx_filename = "evaluation_results_COHERE_bi_and_Cross-encoding.xlsx"
df.to_excel(xlsx_filename, index=False)

print(f"\nMetrics have been exported to {xlsx_filename}")


### BM25 and sentence embeddings

In [ ]:
"""
Zuveir Jameer
8 April 2025
BM25 and reranking with sentence enbeddings

"""

import json
import numpy as np
from elasticsearch import Elasticsearch
from tqdm import tqdm
import time
import pandas as pd
from datetime import datetime

#Evaluation Metric Functions
def precision_at_k(re_ranked_docs, relevant_docs, k):
    top_k_docs = re_ranked_docs[:k]
    num_relevant_in_k = sum(1 for doc in top_k_docs 
                            if doc in relevant_docs and relevant_docs[doc] > 0)
    return num_relevant_in_k / k if k else 0.0

def recall_at_k(re_ranked_docs, relevant_docs, k):
    top_k_docs = re_ranked_docs[:k]
    total_relevant = sum(1 for doc, rel in relevant_docs.items() if rel > 0)
    if total_relevant == 0:
        return 0.0
    num_relevant_in_k = sum(1 for doc in top_k_docs 
                            if doc in relevant_docs and relevant_docs[doc] > 0)
    return num_relevant_in_k / total_relevant

def f1_at_k(re_ranked_docs, relevant_docs, k):
    p = precision_at_k(re_ranked_docs, relevant_docs, k)
    r = recall_at_k(re_ranked_docs, relevant_docs, k)
    if (p + r) == 0:
        return 0.0
    return 2 * (p * r) / (p + r)

def average_precision(re_ranked_docs, relevant_docs):
    if not relevant_docs:
        return 0.0
    
    relevant_indices = [
        i for i, doc in enumerate(re_ranked_docs)
        if doc in relevant_docs and relevant_docs[doc] > 0
    ]
    if not relevant_indices:
        return 0.0
    
    ap_sum = 0.0
    for idx, rank_pos in enumerate(relevant_indices, start=1):
        prec = precision_at_k(re_ranked_docs, relevant_docs, rank_pos+1)
        ap_sum += prec
    return ap_sum / len(relevant_indices)

def ndcg_at_k(re_ranked_docs, relevant_docs, k):
    def dcg(scores):
        return sum(( (2**s - 1) / np.log2(idx + 2)) for idx, s in enumerate(scores))
    
    top_k_docs = re_ranked_docs[:k]
    actual_scores = [relevant_docs[doc] if doc in relevant_docs else 0 for doc in top_k_docs]
    actual_dcg = dcg(actual_scores)
    
    sorted_rels = sorted(relevant_docs.values(), reverse=True)[:k]
    ideal_dcg = dcg(sorted_rels)
    
    if ideal_dcg == 0:
        return 0.0
    return actual_dcg / ideal_dcg

def mrr_at_k(re_ranked_docs, relevant_docs, k):
    for i, doc_id in enumerate(re_ranked_docs[:k], start=1):
        if doc_id in relevant_docs and relevant_docs[doc_id] > 0:
            return 1.0 / i
    return 0.0

# Load queries and qrels from JSONL files
def load_jsonl(filepath):
    data = {}
    with open(filepath, "r") as f:
        for line in f:
            entry = json.loads(line)
            key = entry["query_id"]
            if "text" in entry:
                data[key] = entry["text"].strip()
            else:
                if key not in data:
                    data[key] = {}
                data[key][entry["doc_id"]] = int(entry.get("relevance", 0))
    return data

# Stage 1: initial candidate retrieval based on text fields.
def initial_candidate_retrieval(query_text, candidate_size=500):
    initial_body = {
        "query": {
            "multi_match": {
                "query": query_text,
                "fields": ["title", "summary", "detailed_descrption^2"]
            }
        }
    }
    response = es.search(index="ir_dev_sentence_index", body=initial_body, size=candidate_size)
    print("Time taken to retrieve lexical search): {} ms".format(response["took"]))
    condidate_docs = [hit["_source"]["doc_id"] for hit in response["hits"]["hits"]]
    candidate_ids = [hit["_id"] for hit in response["hits"]["hits"]]
    return candidate_ids, condidate_docs

# Stage 2: vector re-ranking on the candidate set.
def vector_re_ranking(query_text, candidate_ids, top_k=50):
    query_vector = get_embedding(query_text)
    vector_query_body = {
        "query": {
            "script_score": {
                "query": {
                    "bool": {
                        "filter": [
                            {"ids": {"values": candidate_ids}},
                            {"exists": {"field": "detailed_descrption_vector_1"}}
                        ]
                    }
                },
                "script": {
                    "source": "cosineSimilarity(params.query_vector, 'detailed_descrption_vector_1') + 1.0",
                    "params": {"query_vector": query_vector}
                }
            }
        }
    }
    response = es.search(index="ir_dev_sentence_index", body=vector_query_body, size=top_k)
    print(" Time taken to retrieve semantic search: {} ms".format(response["took"]))
    return [hit["_source"]["doc_id"] for hit in response["hits"]["hits"]]

# Load queries and qrels
queries = load_jsonl("trec_ct_2021_query.jsonl")
qrels = load_jsonl("trec_ct_2021_qrels.jsonl")  # Structure: {query_id: {doc_id: relevance, ...}, ...}

# Set the threshold. e.g onsider any document with relevance >= threshold as relevant for evaluation
RELEVANCE_THRESHOLD = 2

def get_relevant_docs(qrel_dict, threshold=1):
    return {doc_id: rel for doc_id, rel in qrel_dict.items() if rel >= threshold}

# For ndcg calculation
def get_all_docs(qrel_dict):
    """Return the entire dictionary of {doc_id: relevance}, including relevance=0."""
    return {doc_id: rel for doc_id, rel in qrel_dict.items()}

# Evaluation loop.
INITIAL_SET=100
K = 10  # top K for evaluation
all_metrics = []

print("Processing queries and computing evaluation metrics...")
for qid, query_text in tqdm(queries.items(), total=len(queries)):
    if not query_text:
        continue

    #Track total time
    start_time = time.time()

    # Stage 1: retrieve candidate document IDs.
    candidate_ids, candidate_docs = initial_candidate_retrieval(query_text, candidate_size=INITIAL_SET)
    print(f"Query {qid} Stage 1 retrieved docs: {candidate_docs}")
    # Stage 2: re-rank candidates via vector search.
    retrieved_docs = vector_re_ranking(query_text, candidate_ids, top_k=K)
    
    # Mark the end time for entire query processing
    total_time = time.time() - start_time

    # Get relevant docs for this query from qrels using the relevance threshold for evaluation
    relevant_docs = get_relevant_docs(qrels.get(qid, {}), threshold=RELEVANCE_THRESHOLD)
    
    #All docs retrieved (i.e no threshold) to compute nDCG based on graded relevance 0,1,2
    ndcg_relevant_docs = get_all_docs(qrels.get(qid, {}))

    # Compute evaluation measures using our functions.
    prec = precision_at_k(retrieved_docs, relevant_docs, K)
    rec = recall_at_k(retrieved_docs, relevant_docs, K)
    f1 = f1_at_k(retrieved_docs, relevant_docs, K)
    ap = average_precision(retrieved_docs, relevant_docs)
    ndcg = ndcg_at_k(retrieved_docs, ndcg_relevant_docs, K)
    mrr_val = mrr_at_k(retrieved_docs, relevant_docs, K)

    metrics = {
        "query_id": qid,
        "precision": prec,
        "recall": rec,
        "f1": f1,
        "average_precision": ap,
        "ndcg": ndcg,
        "mrr": mrr_val,
        "time_seconds": total_time
    }
    all_metrics.append(metrics)
    
    print(f"  Query {qid}: Stage 2 retrieved re-ranked docs {retrieved_docs} Precision@{K}: {prec:.4f}, Recall@{K}: {rec:.4f}, F1@{K}: {f1:.4f}, AP: {ap:.4f}, nDCG@{K}: {ndcg:.4f}, MRR@{K}={mrr_val:.4f}")
    print(f"    Total time for query: {total_time:.2f} seconds")

# Compute overall evaluation metrics.
if all_metrics:
    avg_precision = np.mean([m["precision"] for m in all_metrics])
    avg_recall = np.mean([m["recall"] for m in all_metrics])
    avg_f1 = np.mean([m["f1"] for m in all_metrics])
    mean_ap = np.mean([m["average_precision"] for m in all_metrics])
    mean_ndcg = np.mean([m["ndcg"] for m in all_metrics])
    mean_mrr = np.mean([m["mrr"] for m in all_metrics])
    avg_time = np.mean([m["time_seconds"] for m in all_metrics])
    
    print("\n--- Overall Evaluation (BM25LW and Semantic) ---")
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"Mean Precision@{K}: {avg_precision:.4f}")
    print(f"Mean Recall@{K}: {avg_recall:.4f}")
    print(f"Mean F1@{K}: {avg_f1:.4f}")
    print(f"Mean Average Precision (MAP): {mean_ap:.4f}")
    print(f"Mean nDCG@{K}: {mean_ndcg:.4f}")
    print(f"Mean MRR@{K}: {mean_mrr:.4f}")
    print(f"Average total time per query: {avg_time:.2f} seconds")
else:
    print("No queries processed.")

# Create a DataFrame to export
data = {
    "Test": "BM25LW+Sent.Emd.",
    "Date/Time": [current_time],
    "Relevance Threshold": [RELEVANCE_THRESHOLD],
    f"Mean Precision@{K}": [avg_precision],
    f"Mean Recall@{K}": [avg_recall],
    f"Mean F1@{K}": [avg_f1],
    "Mean Average Precision (MAP)": [mean_ap],
    f"Mean nDCG@{K}": [mean_ndcg],
    f"Mean MRR@{K}": [mean_mrr],
    "Avg Time per Query (s)": [avg_time],
}

df = pd.DataFrame(all_metrics)
df = df[[
    "query_id",
    "precision",
    "recall",
    "f1",
    "average_precision",
    "ndcg",
    "mrr",
    "time_seconds"
]]
# Export to Excel
output_file = "BM25LW_and_sentence_embedding.xlsx"
df.to_excel(output_file, index=False)
print(f"Metrics successfully exported to {output_file}")

# Export final computed results

df_data = pd.DataFrame(data)
# Export to an Excel file
xlsx_filename = "evaluation_results_BM25LW_and_sentence_embeddings.xlsx"
df_data.to_excel(xlsx_filename, index=False)

print(f"\nMetrics have been exported to {xlsx_filename}")

Processing queries and computing evaluation metrics...


  0%|          | 0/75 [00:00<?, ?it/s]/var/folders/b3/59dkzgnj0zxdhdx31d2ztm000000gn/T/ipykernel_24553/3261051846.py:104: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.search(index="ir_dev_sentence_index", body=initial_body, size=candidate_size)


Time taken to retrieve lexical search): 335 ms
Query 1 Stage 1 retrieved docs: ['NCT00968240', 'NCT03633552', 'NCT00089427', 'NCT00003176', 'NCT02453360', 'NCT02942264', 'NCT03528642', 'NCT00976313', 'NCT03896568', 'NCT00003775', 'NCT01466686', 'NCT00360828', 'NCT00004017', 'NCT03513679', 'NCT00004204', 'NCT00613093', 'NCT01256554', 'NCT00047879', 'NCT00003537', 'NCT01784393', 'NCT03434262', 'NCT03948386', 'NCT03743740', 'NCT03902262', 'NCT01913782', 'NCT01502241', 'NCT00007982', 'NCT03174782', 'NCT03022578', 'NCT04686630', 'NCT00064363', 'NCT00790231', 'NCT03224104', 'NCT00841555', 'NCT01812460', 'NCT01847235', 'NCT03647631', 'NCT02174770', 'NCT00052624', 'NCT00004080', 'NCT04193995', 'NCT01597791', 'NCT03511053', 'NCT04771936', 'NCT01004874', 'NCT04458753', 'NCT00362570', 'NCT00526812', 'NCT00006263', 'NCT00006080', 'NCT01678365', 'NCT01220362', 'NCT04222777', 'NCT00039364', 'NCT03641443', 'NCT00004259', 'NCT01456377', 'NCT00473408', 'NCT00564135', 'NCT00028795', 'NCT00301418', 'NCT0

/var/folders/b3/59dkzgnj0zxdhdx31d2ztm000000gn/T/ipykernel_24553/3261051846.py:131: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.search(index="ir_dev_sentence_index", body=vector_query_body, size=top_k)
  1%|▏         | 1/75 [00:01<01:32,  1.25s/it]

 Time taken to retrieve semantic search: 122 ms
  Query 1: Stage 2 retrieved re-ranked docs ['NCT00003470', 'NCT01502241', 'NCT00003537', 'NCT00004259', 'NCT00003775', 'NCT03906448', 'NCT00028795', 'NCT03633552', 'NCT00841555', 'NCT00473408'] Precision@10: 0.2000, Recall@10: 0.0426, F1@10: 0.0702, AP: 0.8333, nDCG@10: 0.5250, MRR@10=1.0000
    Total time for query: 1.25 seconds
Time taken to retrieve lexical search): 180 ms
Query 2 Stage 1 retrieved docs: ['NCT04300686', 'NCT04299971', 'NCT04242225', 'NCT00412386', 'NCT01575249', 'NCT02624934', 'NCT02146755', 'NCT02806310', 'NCT04281771', 'NCT02713932', 'NCT02395107', 'NCT01050361', 'NCT04344353', 'NCT00535899', 'NCT02241109', 'NCT03633383', 'NCT03474159', 'NCT04078867', 'NCT01837160', 'NCT00294775', 'NCT02071849', 'NCT01589380', 'NCT03628313', 'NCT02792452', 'NCT03549052', 'NCT02910349', 'NCT01845285', 'NCT02370511', 'NCT02628899', 'NCT00005199', 'NCT02060071', 'NCT03442400', 'NCT02316587', 'NCT02830204', 'NCT03953755', 'NCT01679431',

...

 Time taken to retrieve semantic search: 3 ms
  Query 73: Stage 2 retrieved re-ranked docs ['NCT02625688', 'NCT03569254', 'NCT03570775', 'NCT00114543', 'NCT00692224', 'NCT02361788', 'NCT04445675', 'NCT02594904', 'NCT03876678', 'NCT03295162'] Precision@10: 0.2000, Recall@10: 0.1000, F1@10: 0.1333, AP: 0.3333, nDCG@10: 0.3412, MRR@10=0.3333
    Total time for query: 0.30 seconds
Time taken to retrieve lexical search): 55 ms
Query 74 Stage 1 retrieved docs: ['NCT00306046', 'NCT03902262', 'NCT00685971', 'NCT00381914', 'NCT00747994', 'NCT04364724', 'NCT03928730', 'NCT02603042', 'NCT02439112', 'NCT01430793', 'NCT02309047', 'NCT00222872', 'NCT01640119', 'NCT03721497', 'NCT00464295', 'NCT03994172', 'NCT04783974', 'NCT00373594', 'NCT01153568', 'NCT04303026', 'NCT03336203', 'NCT00785824', 'NCT02802384', 'NCT01062230', 'NCT00724178', 'NCT02078531', 'NCT00578344', 'NCT02387892', 'NCT03344835', 'NCT01476501', 'NCT02622607', 'NCT00392093', 'NCT00172107', 'NCT02232945', 'NCT01784393', 'NCT01048177', 

 99%|█████████▊| 74/75 [00:24<00:00,  3.40it/s]

 Time taken to retrieve semantic search: 3 ms
  Query 74: Stage 2 retrieved re-ranked docs ['NCT00306046', 'NCT00747994', 'NCT02802384', 'NCT00108394', 'NCT02603042', 'NCT00222872', 'NCT04273490', 'NCT00580281', 'NCT04018287', 'NCT00172107'] Precision@10: 0.3000, Recall@10: 0.2727, F1@10: 0.2857, AP: 1.0000, nDCG@10: 0.4690, MRR@10=1.0000
    Total time for query: 0.29 seconds
Time taken to retrieve lexical search): 42 ms
Query 75 Stage 1 retrieved docs: ['NCT04236843', 'NCT03987269', 'NCT01775501', 'NCT01599546', 'NCT03336203', 'NCT04568005', 'NCT02088866', 'NCT02543905', 'NCT02109952', 'NCT03001050', 'NCT00654563', 'NCT00682994', 'NCT01907334', 'NCT00478296', 'NCT03279991', 'NCT01952834', 'NCT00367627', 'NCT02069366', 'NCT00723567', 'NCT01599975', 'NCT02414529', 'NCT02488200', 'NCT01722578', 'NCT02474940', 'NCT03876652', 'NCT01961219', 'NCT01800604', 'NCT03008005', 'NCT03773809', 'NCT04659265', 'NCT02358876', 'NCT04825626', 'NCT03109808', 'NCT03907514', 'NCT01109849', 'NCT00498719', 

100%|██████████| 75/75 [00:24<00:00,  3.01it/s]

 Time taken to retrieve semantic search: 3 ms
  Query 75: Stage 2 retrieved re-ranked docs ['NCT04858074', 'NCT02358876', 'NCT03876652', 'NCT03585257', 'NCT03732118', 'NCT01599975', 'NCT03420339', 'NCT01601262', 'NCT01722578', 'NCT00654563'] Precision@10: 0.2000, Recall@10: 0.0142, F1@10: 0.0265, AP: 0.3500, nDCG@10: 0.2758, MRR@10=0.5000
    Total time for query: 0.29 seconds

--- Overall Evaluation (BM25LW and Semantic) ---
Mean Precision@10: 0.2320
Mean Recall@10: 0.0410
Mean F1@10: 0.0635
Mean Average Precision (MAP): 0.4704
Mean nDCG@10: 0.3475
Mean MRR@10: 0.5388
Average total time per query: 0.33 seconds
Metrics successfully exported to BM25LW_and_sentence_embedding.xlsx

Metrics have been exported to evaluation_results_BM25LW_and_sentence_embeddings.xlsx


### Bi-encoder and Cross-encoding on selected documents based on precision

In [ ]:
"""
Zuveir Jameer
8 April 2025
Bi-encoder and cross-encoder selectively based on precision threshold
"""

import json
import numpy as np
from elasticsearch import Elasticsearch
from tqdm import tqdm
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer
import time
from datetime import datetime
import pandas as pd

# Setup: Load Models for Cross-encoder for re-ranking
cross_tokenizer = AutoTokenizer.from_pretrained("cross-encoder/ms-marco-MiniLM-L6-v2")
cross_model = AutoModelForSequenceClassification.from_pretrained("cross-encoder/ms-marco-MiniLM-L6-v2")


cross_model.eval()
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
cross_model.to(device)

# Evaluation Metric Functions
def precision_at_k(re_ranked_docs, relevant_docs, k):
    top_k_docs = re_ranked_docs[:k]
    num_relevant_in_k = sum(1 for doc in top_k_docs 
                            if doc in relevant_docs and relevant_docs[doc] > 0)
    return num_relevant_in_k / k if k else 0.0

def recall_at_k(re_ranked_docs, relevant_docs, k):
    top_k_docs = re_ranked_docs[:k]
    total_relevant = sum(1 for doc, rel in relevant_docs.items() if rel > 0)
    if total_relevant == 0:
        return 0.0
    num_relevant_in_k = sum(1 for doc in top_k_docs 
                            if doc in relevant_docs and relevant_docs[doc] > 0)
    return num_relevant_in_k / total_relevant

def f1_at_k(re_ranked_docs, relevant_docs, k):
    p = precision_at_k(re_ranked_docs, relevant_docs, k)
    r = recall_at_k(re_ranked_docs, relevant_docs, k)
    if (p + r) == 0:
        return 0.0
    return 2 * (p * r) / (p + r)

def average_precision(re_ranked_docs, relevant_docs):
    if not relevant_docs:
        return 0.0
    
    relevant_indices = [
        i for i, doc in enumerate(re_ranked_docs)
        if doc in relevant_docs and relevant_docs[doc] > 0
    ]
    if not relevant_indices:
        return 0.0
    
    ap_sum = 0.0
    for idx, rank_pos in enumerate(relevant_indices, start=1):
        prec = precision_at_k(re_ranked_docs, relevant_docs, rank_pos+1)
        ap_sum += prec
    return ap_sum / len(relevant_indices)

def ndcg_at_k(re_ranked_docs, relevant_docs, k):
    def dcg(scores):
        return sum(( (2**s - 1) / np.log2(idx + 2)) for idx, s in enumerate(scores))
    
    top_k_docs = re_ranked_docs[:k]
    actual_scores = [relevant_docs[doc] if doc in relevant_docs else 0 for doc in top_k_docs]
    actual_dcg = dcg(actual_scores)
    
    sorted_rels = sorted(relevant_docs.values(), reverse=True)[:k]
    ideal_dcg = dcg(sorted_rels)
    
    if ideal_dcg == 0:
        return 0.0
    return actual_dcg / ideal_dcg

def mrr_at_k(re_ranked_docs, relevant_docs, k):
    for i, doc_id in enumerate(re_ranked_docs[:k], start=1):
        if doc_id in relevant_docs and relevant_docs[doc_id] > 0:
            return 1.0 / i
    return 0.0

# Data Loading
def load_jsonl(filename):
    with open(filename, "r") as f:
        return [json.loads(line) for line in f]

queries = load_jsonl("trec_ct_2021_query.jsonl")
qrels_list = load_jsonl("trec_ct_2021_qrels.jsonl")

# Create a dictionary mapping query_id to {doc_id: relevance}
qrels = {}
for q in qrels_list:
    qid = q["query_id"]
    if qid not in qrels:
        qrels[qid] = {}
    relevance = int(q.get("relevance", 0))
    qrels[qid][q["doc_id"]] = relevance

print(f"Loaded {len(queries)} queries")
print(f"Loaded qrels for {len(qrels)} queries")

RELEVANCE_THRESHOLD = 2

filtered_qrels = {
    qid: {doc: rel for doc, rel in rels.items() if rel >= RELEVANCE_THRESHOLD}
    for qid, rels in qrels.items()
}

ndcg_qrels = {
    qid: {doc: rel for doc, rel in rels.items()}
    for qid, rels in qrels.items()
}

# Dense Retrieval (Stage 1) Functions using Bi-Encoder
def search_detailed_descrption(query_vector, K=100):
    """
    Retrieve candidate documents using dense vector search with cosine similarity.
    Each document in ES has a pre-computed vector in the field 
    'detailed_descrption_vector_1'.
    """
    search_body = {
      "query": {
        "script_score": {
          "query": {
            "bool": {
              "must": [
                {"match_all": {}},
                {"exists": {"field": "detailed_descrption_vector_1"}}
              ]
            }
          },
          "script": {
            "source": "cosineSimilarity(params.query_vector, 'detailed_descrption_vector_1') + 1.0",
            "params": {"query_vector": query_vector}
          }
        }
      }
    }
    response = es.search(index="ir_dev_sentence_index", body=search_body, size=K) 
    #size=K: Limits the number of results returned to K documents. The query will only return up to K matching documents.
    return [hit["_source"]["doc_id"] for hit in response['hits']['hits']]

def get_document_text(doc_id):
    """
    Retrieve the detailed description for a document given its ID.
    Make sure that the field name matches your index mapping.
    """
    query = {
        "query": {
            "term": {"doc_id.keyword": doc_id}
        }
    }
    response = es.search(index="ir_dev_sentence_index", body=query, size=1)
    hits = response.get("hits", {}).get("hits", [])
    if hits:
        doc_text = hits[0]["_source"].get("detailed_descrption", "").strip()
        # Debug: Uncomment the following line to inspect retrieved text
        # print(f"Doc {doc_id} text snippet: {doc_text[:100]}")
        return doc_text if doc_text else None
    else:
        return None

K = 20  # Number of candidates from dense retrieval
reRank_K=20 # Number of results from re-ranker
top_k = 10  # documents to be processed for results
selectivePrecision = 0.3  #Only records from bi-encoder with precision below or equal to this threshold are sent to cross-encoder.


def cross_encoder_rerank(query_text, candidate_doc_ids, top_k=reRank_K):
    candidate_pairs = []
    valid_doc_ids = []
    for doc_id in candidate_doc_ids:
        try:
            doc_text = get_document_text(doc_id)
            if doc_text is None or not doc_text.strip():
                continue
            candidate_pairs.append((query_text, doc_text))
            valid_doc_ids.append(doc_id)
        except Exception as e:
            print(f"Error fetching doc {doc_id}: {e}")
            continue

    if not candidate_pairs:
        print("No valid candidate pairs found; returning original candidates.")
        return candidate_doc_ids

    inputs = cross_tokenizer.batch_encode_plus(
        candidate_pairs, 
        padding=True, 
        truncation=True, 
        return_tensors="pt"
    )
    inputs = {k: v.to(cross_model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = cross_model(**inputs)

    if outputs.logits.shape[-1] == 1:
        scores = outputs.logits.squeeze(-1)
    else:
        scores = torch.softmax(outputs.logits, dim=1)[:, 1]

    ranked = sorted(
        zip(valid_doc_ids, scores.tolist()), 
        key=lambda x: x[1], 
        reverse=True
    )

    # If top_k is specified, slice the top_k
    if top_k is not None:
        ranked = ranked[:top_k]

    # Return just the document IDs, in ranked order
    return [doc_id for doc_id, score in ranked]

all_metrics = []

print("Processing queries and retrieving documents...")
for query in tqdm(queries, total=len(queries)):
    qid = query["query_id"]
    query_text = query["text"]
    if not query_text.strip():
        print(f"Query {qid} is empty, skipping.")
        continue
    
    start_time = time.time()
    
    # Stage 1: Get the dense vector for the query and retrieve candidates
    query_vector = get_embedding(query_text)
    try:
        retrieved_docs = search_detailed_descrption(query_vector, K)
        print(f"Query {qid} Stage 1 retrieved docs: {retrieved_docs}")
    except Exception as e:
        print(f"Error retrieving documents for query {qid}: {e}")
        continue

  
    # Evaluate bi-encoder precision before deciding on cross-encoder re-ranking
    bi_precision = precision_at_k(retrieved_docs, filtered_qrels.get(qid, {}), top_k)
    if bi_precision <= selectivePrecision:
        print(f"Query {qid}: Bi-encoder precision is low ({bi_precision:.4f}), applying cross-encoder re-ranking")
        try:
            re_ranked_docs = cross_encoder_rerank(query_text, retrieved_docs)
        except Exception as e:
            print(f"Error in cross encoder re-ranking for query {qid}: {e}")
            re_ranked_docs = retrieved_docs
    else:
        print(f"Query {qid}: Bi-encoder precision is high ({bi_precision:.4f}), skipping cross-encoder re-ranking")
        re_ranked_docs = retrieved_docs

    total_time = time.time() - start_time
    
    # Evaluate using filtered qrels
    relevant = filtered_qrels.get(qid, {})
    ndcg_relevant = ndcg_qrels.get(qid, {})
    p10 = precision_at_k(re_ranked_docs, relevant, top_k)
    r10 = recall_at_k(re_ranked_docs, relevant, top_k)
    f1 = f1_at_k(re_ranked_docs, relevant, top_k)
    ap = average_precision(re_ranked_docs, relevant)
    ndcg = ndcg_at_k(re_ranked_docs, ndcg_relevant, top_k)
    mrr_val = mrr_at_k(re_ranked_docs, relevant, top_k)

    all_metrics.append({
        "query_id": qid, 
        "precision": p10, 
        "recall": r10, 
        "f1": f1, 
        "ap": ap, 
        "ndcg": ndcg,
        "mrr": mrr_val,
        "time_seconds": total_time
    })
    print(f"  Query {qid} stage 2 retrieved (after processing): {re_ranked_docs}")
    print(f"   Query {qid} metrics: Precision@{top_k}: {p10:.4f}, Recall@{top_k}: {r10:.4f}, F1@{top_k}: {f1:.4f}, AP: {ap:.4f}, nDCG@{top_k}: {ndcg:.4f}, MRR@{top_k}={mrr_val:.4f}")
    print(f"    Total time for query: {total_time:.2f} seconds")
    
if all_metrics:
    avg_precision = np.mean([m["precision"] for m in all_metrics])
    avg_recall = np.mean([m["recall"] for m in all_metrics])
    avg_f1 = np.mean([m["f1"] for m in all_metrics])
    avg_ap = np.mean([m["ap"] for m in all_metrics])
    avg_ndcg = np.mean([m["ndcg"] for m in all_metrics])
    avg_mrr = np.mean([m["mrr"] for m in all_metrics])
    avg_time = np.mean([m["time_seconds"] for m in all_metrics])

    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"Date/time:{current_time}")
    print("\nBi-encoder and cross-encoder - Overall evaluation metrics:")
    print(f"Relevance Threshold: {RELEVANCE_THRESHOLD}")
    print("Mean Precision@{}: {:.4f}".format(top_k, avg_precision))
    print("Mean Recall@{}: {:.4f}".format(top_k, avg_recall))
    print("Mean F1@{}: {:.4f}".format(top_k, avg_f1))
    print("Mean Average Precision (MAP): {:.4f}".format(avg_ap))
    print("Mean nDCG@{}: {:.4f}".format(top_k, avg_ndcg))
    print(f"Mean MRR{top_k}: {avg_mrr:.4f}")
    print(f"Average total time per query: {avg_time:.2f} seconds")
else:
    print("No query metrics to average.")

# Export the computed results for all queries
df = pd.DataFrame(all_metrics)
df = df[[
    "query_id",
    "precision",
    "recall",
    "f1",
    "ap",
    "ndcg",
    "mrr",
    "time_seconds"
]]
# Export to Excel
output_file = "sentence_embedding_bi_and_Cross-encoding_metrics_output.xlsx"
df.to_excel(output_file, index=False)
print(f"Metrics successfully exported to {output_file}")

# Export final computed results
data = {
    "Test": f"Bi-encoder + Cross-encoder (precision <= {selectivePrecision})",
    "Date/Time": [current_time],
    "Relevance Threshold": [RELEVANCE_THRESHOLD],
    f"Mean Precision@{top_k}": [avg_precision],
    f"Mean Recall@{top_k}": [avg_recall],
    f"Mean F1@{top_k}": [avg_f1],
    "Mean Average Precision (MAP)": [avg_ap],
    f"Mean nDCG@{top_k}": [avg_ndcg],
    f"Mean MRR@{top_k}": [avg_mrr],
    "Avg Time per Query (s)": [avg_time],
}
df = pd.DataFrame(data)

# Export to an Excel file
xlsx_filename = f"evaluation_results_sentence_embeddings_bi_and_Cross-encoding_Selective_{selectivePrecision}.xlsx"
df.to_excel(xlsx_filename, index=False)

print(f"\nMetrics have been exported to {xlsx_filename}")


Loaded 75 queries
Loaded qrels for 75 queries
Processing queries and retrieving documents...


  0%|          | 0/75 [00:00<?, ?it/s]/var/folders/b3/59dkzgnj0zxdhdx31d2ztm000000gn/T/ipykernel_54942/2646809909.py:145: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.search(index="ir_dev_sentence_index", body=search_body, size=K)
  1%|▏         | 1/75 [00:00<00:19,  3.86it/s]

Query 1 Stage 1 retrieved docs: ['NCT00003470', 'NCT01502241', 'NCT00003537', 'NCT00003621', 'NCT00003471', 'NCT00004259', 'NCT00783393', 'NCT00003775', 'NCT03906448', 'NCT00033280', 'NCT02755987', 'NCT00028795', 'NCT03633552', 'NCT00841555', 'NCT00003375', 'NCT00473408', 'NCT00002569', 'NCT00003465', 'NCT00360828', 'NCT03022578']
Query 1: Bi-encoder precision is high (0.4000), skipping cross-encoder re-ranking
  Query 1 stage 2 retrieved (after processing): ['NCT00003470', 'NCT01502241', 'NCT00003537', 'NCT00003621', 'NCT00003471', 'NCT00004259', 'NCT00783393', 'NCT00003775', 'NCT03906448', 'NCT00033280', 'NCT02755987', 'NCT00028795', 'NCT03633552', 'NCT00841555', 'NCT00003375', 'NCT00473408', 'NCT00002569', 'NCT00003465', 'NCT00360828', 'NCT03022578']
   Query 1 metrics: Precision@10: 0.4000, Recall@10: 0.0851, F1@10: 0.1404, AP: 0.8042, nDCG@10: 0.6502, MRR@10=1.0000
    Total time for query: 0.26 seconds
Query 2 Stage 1 retrieved docs: ['NCT03628313', 'NCT02753023', 'NCT02395107', 

/var/folders/b3/59dkzgnj0zxdhdx31d2ztm000000gn/T/ipykernel_54942/2646809909.py:159: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.search(index="ir_dev_sentence_index", body=query, size=1)
  3%|▎         | 2/75 [00:01<00:59,  1.22it/s]

  Query 2 stage 2 retrieved (after processing): ['NCT03628313', 'NCT01980797', 'NCT00590135', 'NCT02316587', 'NCT02395107', 'NCT02344199', 'NCT02531581', 'NCT02792452', 'NCT03863132', 'NCT04344353', 'NCT00624884', 'NCT01575249', 'NCT02010177', 'NCT04140019', 'NCT02753023', 'NCT01157572', 'NCT04144725', 'NCT03867708']
   Query 2 metrics: Precision@10: 0.4000, Recall@10: 0.0290, F1@10: 0.0541, AP: 0.7069, nDCG@10: 0.6045, MRR@10=1.0000
    Total time for query: 1.21 seconds
Query 3 Stage 1 retrieved docs: ['NCT03995823', 'NCT02098252', 'NCT04569279', 'NCT00363662', 'NCT04329208', 'NCT01158807', 'NCT02394678', 'NCT04037267', 'NCT03532165', 'NCT03291652', 'NCT04572568', 'NCT04567277', 'NCT02775318', 'NCT03150524', 'NCT03691870', 'NCT00235248', 'NCT03477188', 'NCT00029315', 'NCT03676868', 'NCT01856699']
Query 3: Bi-encoder precision is low (0.3000), applying cross-encoder re-ranking


...

  Query 73 stage 2 retrieved (after processing): ['NCT04215601', 'NCT02774434', 'NCT02446951', 'NCT01434810', 'NCT01622699', 'NCT03942757', 'NCT04099602', 'NCT02625688', 'NCT01136577', 'NCT03183986', 'NCT04719104', 'NCT00653874', 'NCT02613676', 'NCT03055481', 'NCT03570775', 'NCT03569254']
   Query 73 metrics: Precision@10: 0.4000, Recall@10: 0.2000, F1@10: 0.2667, AP: 0.4267, nDCG@10: 0.5321, MRR@10=0.5000
    Total time for query: 0.86 seconds


 99%|█████████▊| 74/75 [00:45<00:00,  1.79it/s]

Query 74 Stage 1 retrieved docs: ['NCT00306046', 'NCT00051636', 'NCT00774020', 'NCT00480662', 'NCT00747994', 'NCT04222452', 'NCT02106455', 'NCT00894075', 'NCT02802384', 'NCT02056730', 'NCT00453128', 'NCT03993821', 'NCT03859895', 'NCT00108394', 'NCT00195936', 'NCT00066183', 'NCT00417612', 'NCT02198001', 'NCT02967042', 'NCT00973336']
Query 74: Bi-encoder precision is high (0.7000), skipping cross-encoder re-ranking
  Query 74 stage 2 retrieved (after processing): ['NCT00306046', 'NCT00051636', 'NCT00774020', 'NCT00480662', 'NCT00747994', 'NCT04222452', 'NCT02106455', 'NCT00894075', 'NCT02802384', 'NCT02056730', 'NCT00453128', 'NCT03993821', 'NCT03859895', 'NCT00108394', 'NCT00195936', 'NCT00066183', 'NCT00417612', 'NCT02198001', 'NCT02967042', 'NCT00973336']
   Query 74 metrics: Precision@10: 0.7000, Recall@10: 0.6364, F1@10: 0.6667, AP: 0.9478, nDCG@10: 0.7885, MRR@10=1.0000
    Total time for query: 0.25 seconds
Query 75 Stage 1 retrieved docs: ['NCT00443768', 'NCT00001928', 'NCT002030

100%|██████████| 75/75 [00:46<00:00,  1.61it/s]

  Query 75 stage 2 retrieved (after processing): ['NCT00001928', 'NCT03568968', 'NCT00443768', 'NCT00607451', 'NCT00199394', 'NCT00307450', 'NCT00472355', 'NCT04472130', 'NCT03433950', 'NCT00199420', 'NCT00199355', 'NCT00199407', 'NCT03956979', 'NCT00526630', 'NCT00666653', 'NCT00148512']
   Query 75 metrics: Precision@10: 0.3000, Recall@10: 0.0213, F1@10: 0.0397, AP: 0.7917, nDCG@10: 0.5685, MRR@10=1.0000
    Total time for query: 0.83 seconds
Date/time:2025-04-10 08:24:20

Bi-encoder and cross-encoder - Overall evaluation metrics:
Relevance Threshold: 2
Mean Precision@10: 0.3333
Mean Recall@10: 0.0740
Mean F1@10: 0.1074
Mean Average Precision (MAP): 0.4595
Mean nDCG@10: 0.4379
Mean MRR10: 0.5775
Average total time per query: 0.62 seconds


### Bi-encoder and Cross-encoding on selected documents based on Similarity Threshold

In [ ]:
"""
Zuveir Jameer
2 April 2025
Bi-encoder and cross-encoder selective based in similarity threshold
"""

import json
import numpy as np
from elasticsearch import Elasticsearch
from tqdm import tqdm
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer
import time
from datetime import datetime
import pandas as pd

# Load Model: Cross-encoder for re-ranking

cross_tokenizer = AutoTokenizer.from_pretrained("cross-encoder/ms-marco-MiniLM-L6-v2")
cross_model = AutoModelForSequenceClassification.from_pretrained("cross-encoder/ms-marco-MiniLM-L6-v2")

cross_model.eval()
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
cross_model.to(device)

# Evaluation Metric Functions
def precision_at_k(re_ranked_docs, relevant_docs, k):
    top_k_docs = re_ranked_docs[:k]
    num_relevant_in_k = sum(1 for doc in top_k_docs 
                            if doc in relevant_docs and relevant_docs[doc] > 0)
    return num_relevant_in_k / k if k else 0.0

def recall_at_k(re_ranked_docs, relevant_docs, k):
    top_k_docs = re_ranked_docs[:k]
    total_relevant = sum(1 for doc, rel in relevant_docs.items() if rel > 0)
    if total_relevant == 0:
        return 0.0
    num_relevant_in_k = sum(1 for doc in top_k_docs 
                            if doc in relevant_docs and relevant_docs[doc] > 0)
    return num_relevant_in_k / total_relevant

def f1_at_k(re_ranked_docs, relevant_docs, k):
    p = precision_at_k(re_ranked_docs, relevant_docs, k)
    r = recall_at_k(re_ranked_docs, relevant_docs, k)
    if (p + r) == 0:
        return 0.0
    return 2 * (p * r) / (p + r)

def average_precision(re_ranked_docs, relevant_docs):
    if not relevant_docs:
        return 0.0
    
    relevant_indices = [
        i for i, doc in enumerate(re_ranked_docs)
        if doc in relevant_docs and relevant_docs[doc] > 0
    ]
    if not relevant_indices:
        return 0.0
    
    ap_sum = 0.0
    for idx, rank_pos in enumerate(relevant_indices, start=1):
        prec = precision_at_k(re_ranked_docs, relevant_docs, rank_pos+1)
        ap_sum += prec
    return ap_sum / len(relevant_indices)

def ndcg_at_k(re_ranked_docs, relevant_docs, k):
    def dcg(scores):
        return sum(((2**s - 1) / np.log2(idx + 2)) for idx, s in enumerate(scores))
    
    top_k_docs = re_ranked_docs[:k]
    actual_scores = [relevant_docs[doc] if doc in relevant_docs else 0 for doc in top_k_docs]
    actual_dcg = dcg(actual_scores)
    
    sorted_rels = sorted(relevant_docs.values(), reverse=True)[:k]
    ideal_dcg = dcg(sorted_rels)
    
    if ideal_dcg == 0:
        return 0.0
    return actual_dcg / ideal_dcg

def mrr_at_k(re_ranked_docs, relevant_docs, k):
    for i, doc_id in enumerate(re_ranked_docs[:k], start=1):
        if doc_id in relevant_docs and relevant_docs[doc_id] > 0:
            return 1.0 / i
    return 0.0

# Data Loading
def load_jsonl(filename):
    with open(filename, "r") as f:
        return [json.loads(line) for line in f]

queries = load_jsonl("trec_ct_2021_query.jsonl")
qrels_list = load_jsonl("trec_ct_2021_qrels.jsonl")

# Create a dictionary mapping query_id to {doc_id: relevance}
qrels = {}
for q in qrels_list:
    qid = q["query_id"]
    if qid not in qrels:
        qrels[qid] = {}
    relevance = int(q.get("relevance", 0))
    qrels[qid][q["doc_id"]] = relevance

print(f"Loaded {len(queries)} queries")
print(f"Loaded qrels for {len(qrels)} queries")

RELEVANCE_THRESHOLD = 2

filtered_qrels = {
    qid: {doc: rel for doc, rel in rels.items() if rel >= RELEVANCE_THRESHOLD}
    for qid, rels in qrels.items()
}

ndcg_qrels = {
    qid: {doc: rel for doc, rel in rels.items()}
    for qid, rels in qrels.items()
}


# Dense Retrieval (Stage 1)
def search_detailed_descrption(query_vector, K=100):
    """
    Retrieve candidate documents using dense vector search with cosine similarity.
    Each document in ES has a pre-computed vector in the field 
    'detailed_descrption_vector_2'.
    
    Returns a list of tuples: (doc_id, similarity_score)
    """
    search_body = {
      "query": {
        "script_score": {
          "query": {
            "bool": {
              "must": [
                {"match_all": {}},
                {"exists": {"field": "detailed_descrption_vector_1"}}
              ]
            }
          },
          "script": {
            "source": "cosineSimilarity(params.query_vector, 'detailed_descrption_vector_1') + 1.0",
            "params": {"query_vector": query_vector}
          }
        }
      }
    }
    response = es.search(index="ir_dev_sentence_index", body=search_body, size=K)
    results = []
    for hit in response['hits']['hits']:
        doc_id = hit["_source"]["doc_id"]
        score = hit["_score"]
        results.append((doc_id, score))
    return results

def get_document_text(doc_id):
    """
    Retrieve the detailed description for a document given its ID.
    Make sure that the field name matches your index mapping.
    """
    query = {
        "query": {
            "term": {"doc_id.keyword": doc_id}
        }
    }
    response = es.search(index="ir_dev_sentence_index", body=query, size=1)
    hits = response.get("hits", {}).get("hits", [])
    if hits:
        doc_text = hits[0]["_source"].get("detailed_descrption", "").strip()
        return doc_text if doc_text else None
    else:
        return None

K = 40  # Number of candidates from dense retrieval
top_k = 10  # Number of results from re-ranker

def cross_encoder_rerank(query_text, candidate_doc_ids, top_k=top_k):
    candidate_pairs = []
    valid_doc_ids = []
    for doc_id in candidate_doc_ids:
        try:
            doc_text = get_document_text(doc_id)
            if doc_text is None or not doc_text.strip():
                continue
            candidate_pairs.append((query_text, doc_text))
            valid_doc_ids.append(doc_id)
        except Exception as e:
            print(f"Error fetching doc {doc_id}: {e}")
            continue

    if not candidate_pairs:
        print("No valid candidate pairs found; returning original candidates.")
        return candidate_doc_ids

    inputs = cross_tokenizer.batch_encode_plus(
        candidate_pairs, 
        padding=True, 
        truncation=True, 
        return_tensors="pt"
    )
    inputs = {k: v.to(cross_model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = cross_model(**inputs)

    # Handle both regression and classification styles
    if outputs.logits.shape[-1] == 1:
        scores = outputs.logits.squeeze(-1)
    else:
        scores = torch.softmax(outputs.logits, dim=1)[:, 1]

    ranked = sorted(
        zip(valid_doc_ids, scores.tolist()), 
        key=lambda x: x[1], 
        reverse=True
    )

    if top_k is not None:
        ranked = ranked[:top_k]

    return [doc_id for doc_id, score in ranked]


# Main Processing Loop

all_metrics = []

# Define a threshold on the bi-encoder’s similarity score.
# Note: Since the script adds 1.0 to cosine similarity (range [-1,1]),
# scores range from 0 to 2. Hyperparameter to tune.
similarity_threshold = 1.6 

print("Processing queries and retrieving documents...")
for query in tqdm(queries, total=len(queries)):
    qid = query["query_id"]
    query_text = query["text"]
    if not query_text.strip():
        print(f"Query {qid} is empty, skipping.")
        continue
    
    start_time = time.time()
    
    # Stage 1: Get the dense vector for the query and retrieve candidates
    query_vector = get_embedding(query_text) 
    try:
        # Now retrieved_docs_with_scores is a list of tuples: (doc_id, score)
        retrieved_docs_with_scores = search_detailed_descrption(query_vector, K)
        if not retrieved_docs_with_scores:
            print(f"No documents retrieved for query {qid}")
            continue
        # Separate doc_ids and scores
        retrieved_docs = [doc for doc, score in retrieved_docs_with_scores]
        top_bi_score = retrieved_docs_with_scores[0][1]
        print(f"Query {qid} Stage 1 retrieved docs: {retrieved_docs}")
        print(f"Query {qid} top bi-encoder score: {top_bi_score:.4f}")
    except Exception as e:
        print(f"Error retrieving documents for query {qid}: {e}")
        continue

    # Stage 2: Decide based on similarity threshold whether to apply cross-encoder based on bi-encoder top score.
    if top_bi_score < similarity_threshold:
        print(f"Query {qid}: Top bi-encoder score {top_bi_score:.4f} is below threshold {similarity_threshold}, applying cross-encoder re-ranking")
        try:
            re_ranked_docs = cross_encoder_rerank(query_text, retrieved_docs)
        except Exception as e:
            print(f"Error in cross encoder re-ranking for query {qid}: {e}")
            re_ranked_docs = retrieved_docs
    else:
        print(f"Query {qid}: Top bi-encoder score {top_bi_score:.4f} exceeds threshold {similarity_threshold}, skipping cross-encoder re-ranking")
        re_ranked_docs = retrieved_docs

    total_time = time.time() - start_time
    
    # Evaluate using filtered qrels (ensure your evaluation functions are correct)
    relevant = filtered_qrels.get(qid, {})
    ndcg_relevant = ndcg_qrels.get(qid, {})
    p10 = precision_at_k(re_ranked_docs, relevant, top_k)
    r10 = recall_at_k(re_ranked_docs, relevant, top_k)
    f1 = f1_at_k(re_ranked_docs, relevant, top_k)
    ap = average_precision(re_ranked_docs, relevant)
    ndcg = ndcg_at_k(re_ranked_docs, ndcg_relevant, top_k)
    mrr_val = mrr_at_k(re_ranked_docs, relevant, top_k)

    all_metrics.append({
        "query_id": qid, 
        "precision": p10, 
        "recall": r10, 
        "f1": f1, 
        "ap": ap, 
        "ndcg": ndcg,
        "mrr": mrr_val,
        "time_seconds": total_time
    })
    print(f"  Query {qid} stage 2 retrieved (final): {re_ranked_docs}")
    print(f"   Query {qid} metrics: Precision@{top_k}: {p10:.4f}, Recall@{top_k}: {r10:.4f}, F1@{top_k}: {f1:.4f}, AP: {ap:.4f}, nDCG@{top_k}: {ndcg:.4f}, MRR@{top_k}={mrr_val:.4f}")
    print(f"    Total time for query: {total_time:.2f} seconds")
    
if all_metrics:
    avg_precision = np.mean([m["precision"] for m in all_metrics])
    avg_recall = np.mean([m["recall"] for m in all_metrics])
    avg_f1 = np.mean([m["f1"] for m in all_metrics])
    avg_ap = np.mean([m["ap"] for m in all_metrics])
    avg_ndcg = np.mean([m["ndcg"] for m in all_metrics])
    avg_mrr = np.mean([m["mrr"] for m in all_metrics])
    avg_time = np.mean([m["time_seconds"] for m in all_metrics])

    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"Date/time:{current_time}")
    print("\nBi-encoder and cross-encoder - Overall evaluation metrics:")
    print(f"Relevance Threshold: {RELEVANCE_THRESHOLD}")
    print("Mean Precision@{}: {:.4f}".format(top_k, avg_precision))
    print("Mean Recall@{}: {:.4f}".format(top_k, avg_recall))
    print("Mean F1@{}: {:.4f}".format(top_k, avg_f1))
    print("Mean Average Precision (MAP): {:.4f}".format(avg_ap))
    print("Mean nDCG@{}: {:.4f}".format(top_k, avg_ndcg))
    print(f"Mean MRR{top_k}: {avg_mrr:.4f}")
    print(f"Average total time per query: {avg_time:.2f} seconds")
else:
    print("No query metrics to average.")

# Export the computed results for each query
df = pd.DataFrame(all_metrics)
df = df[[
    "query_id",
    "precision",
    "recall",
    "f1",
    "ap",
    "ndcg",
    "mrr",
    "time_seconds"
]]
output_file = f"sentence_embedding_bi_and_Cross-encoding_metrics_similarity {similarity_threshold} Threshold.xlsx"
df.to_excel(output_file, index=False)
print(f"Metrics successfully exported to {output_file}")

# Export final computed results
data = {
    "Test": "Bi-encoder + Cross-encoder",
    "Date/Time": [current_time],
    "Relevance Threshold": [RELEVANCE_THRESHOLD],
    f"Mean Precision@{top_k}": [avg_precision],
    f"Mean Recall@{top_k}": [avg_recall],
    f"Mean F1@{top_k}": [avg_f1],
    "Mean Average Precision (MAP)": [avg_ap],
    f"Mean nDCG@{top_k}": [avg_ndcg],
    f"Mean MRR@{top_k}": [avg_mrr],
    "Avg Time per Query (s)": [avg_time],
}
df = pd.DataFrame(data)
xlsx_filename = f"evaluation_results_sentence_embeddings_bi_and_Cross-encoding_similarity {similarity_threshold} Threshold .xlsx"
df.to_excel(xlsx_filename, index=False)
print(f"\nMetrics have been exported to {xlsx_filename}")


Loaded 75 queries
Loaded qrels for 75 queries
Processing queries and retrieving documents...


  0%|          | 0/75 [00:00<?, ?it/s]/var/folders/b3/59dkzgnj0zxdhdx31d2ztm000000gn/T/ipykernel_54942/2394684473.py:148: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.search(index="ir_dev_sentence_index", body=search_body, size=K)
  1%|▏         | 1/75 [00:00<00:18,  4.05it/s]

Query 1 Stage 1 retrieved docs: ['NCT00003470', 'NCT01502241', 'NCT00003537', 'NCT00003621', 'NCT00003471', 'NCT00004259', 'NCT00783393', 'NCT00003775', 'NCT03906448', 'NCT00033280', 'NCT02755987', 'NCT00028795', 'NCT03633552', 'NCT00841555', 'NCT00003375', 'NCT00473408', 'NCT00002569', 'NCT00003465', 'NCT00360828', 'NCT03022578', 'NCT00004068', 'NCT00004017', 'NCT03668522', 'NCT03641443', 'NCT00362570', 'NCT00003842', 'NCT01847235', 'NCT00003176', 'NCT00717210', 'NCT00360945', 'NCT00301418', 'NCT00761280', 'NCT00400816', 'NCT00631670', 'NCT00052624', 'NCT00430911', 'NCT00007982', 'NCT00978458', 'NCT03528642', 'NCT03224104']
Query 1 top bi-encoder score: 1.7383
Query 1: Top bi-encoder score 1.7383 exceeds threshold 1.6, skipping cross-encoder re-ranking
  Query 1 stage 2 retrieved (final): ['NCT00003470', 'NCT01502241', 'NCT00003537', 'NCT00003621', 'NCT00003471', 'NCT00004259', 'NCT00783393', 'NCT00003775', 'NCT03906448', 'NCT00033280', 'NCT02755987', 'NCT00028795', 'NCT03633552', 'NC

  3%|▎         | 2/75 [00:00<00:16,  4.30it/s]

Query 2 Stage 1 retrieved docs: ['NCT03628313', 'NCT02753023', 'NCT02395107', 'NCT03332745', 'NCT01157572', 'NCT02010177', 'NCT04061213', 'NCT03863132', 'NCT02792452', 'NCT00590135', 'NCT02531581', 'NCT04140019', 'NCT01980797', 'NCT01575249', 'NCT00624884', 'NCT02316587', 'NCT04344353', 'NCT02344199', 'NCT03867708', 'NCT04144725', 'NCT01419015', 'NCT01458860', 'NCT00294775', 'NCT01845285', 'NCT02624934', 'NCT01275339', 'NCT00699452', 'NCT04444024', 'NCT02283970', 'NCT01756222', 'NCT02146755', 'NCT01636427', 'NCT02448485', 'NCT04026542', 'NCT02481258', 'NCT03049995', 'NCT02806310', 'NCT01579058', 'NCT00176410', 'NCT02241109']
Query 2 top bi-encoder score: 1.7217
Query 2: Top bi-encoder score 1.7217 exceeds threshold 1.6, skipping cross-encoder re-ranking
  Query 2 stage 2 retrieved (final): ['NCT03628313', 'NCT02753023', 'NCT02395107', 'NCT03332745', 'NCT01157572', 'NCT02010177', 'NCT04061213', 'NCT03863132', 'NCT02792452', 'NCT00590135', 'NCT02531581', 'NCT04140019', 'NCT01980797', 'NC

...

Query 73 Stage 1 retrieved docs: ['NCT01136577', 'NCT04215601', 'NCT02625688', 'NCT04099602', 'NCT04719104', 'NCT04418180', 'NCT03942757', 'NCT03055481', 'NCT02154165', 'NCT02446951', 'NCT03183986', 'NCT03569254', 'NCT03570775', 'NCT00653874', 'NCT02613676', 'NCT02774434', 'NCT01434810', 'NCT01622699', 'NCT00635375', 'NCT04365998', 'NCT02372071', 'NCT02372058', 'NCT00114543', 'NCT01203410', 'NCT04021927', 'NCT02805296', 'NCT01470820', 'NCT02156050', 'NCT00735319', 'NCT03866213', 'NCT02356978', 'NCT03599258', 'NCT01550627', 'NCT02612727', 'NCT00692224', 'NCT04585828', 'NCT04251286', 'NCT01762189', 'NCT00288600', 'NCT03130517']
Query 73 top bi-encoder score: 1.8123
Query 73: Top bi-encoder score 1.8123 exceeds threshold 1.6, skipping cross-encoder re-ranking
  Query 73 stage 2 retrieved (final): ['NCT01136577', 'NCT04215601', 'NCT02625688', 'NCT04099602', 'NCT04719104', 'NCT04418180', 'NCT03942757', 'NCT03055481', 'NCT02154165', 'NCT02446951', 'NCT03183986', 'NCT03569254', 'NCT03570775',

 99%|█████████▊| 74/75 [00:21<00:00,  4.29it/s]

Query 74 Stage 1 retrieved docs: ['NCT00306046', 'NCT00051636', 'NCT00774020', 'NCT00480662', 'NCT00747994', 'NCT04222452', 'NCT02106455', 'NCT00894075', 'NCT02802384', 'NCT02056730', 'NCT00453128', 'NCT03993821', 'NCT03859895', 'NCT00108394', 'NCT00195936', 'NCT00066183', 'NCT00417612', 'NCT02198001', 'NCT02967042', 'NCT00973336', 'NCT01875458', 'NCT00668200', 'NCT02603042', 'NCT00415584', 'NCT04184661', 'NCT00712985', 'NCT00324714', 'NCT01839383', 'NCT03716128', 'NCT00377312', 'NCT00159419', 'NCT01306656', 'NCT00572299', 'NCT01222026', 'NCT01360099', 'NCT01419028', 'NCT02235493', 'NCT00222872', 'NCT03713671', 'NCT03027349']
Query 74 top bi-encoder score: 1.8162
Query 74: Top bi-encoder score 1.8162 exceeds threshold 1.6, skipping cross-encoder re-ranking
  Query 74 stage 2 retrieved (final): ['NCT00306046', 'NCT00051636', 'NCT00774020', 'NCT00480662', 'NCT00747994', 'NCT04222452', 'NCT02106455', 'NCT00894075', 'NCT02802384', 'NCT02056730', 'NCT00453128', 'NCT03993821', 'NCT03859895',

100%|██████████| 75/75 [00:21<00:00,  3.45it/s]

Query 75 Stage 1 retrieved docs: ['NCT00443768', 'NCT00001928', 'NCT00203060', 'NCT00199394', 'NCT04380142', 'NCT00526630', 'NCT00607451', 'NCT00148512', 'NCT00472355', 'NCT00666653', 'NCT00307450', 'NCT03956979', 'NCT00199407', 'NCT03433950', 'NCT04379050', 'NCT04750226', 'NCT04472130', 'NCT00199420', 'NCT00199355', 'NCT03568968', 'NCT01003002', 'NCT00466167', 'NCT00004733', 'NCT01970813', 'NCT02424708', 'NCT04687878', 'NCT03703570', 'NCT00794313', 'NCT03011476', 'NCT00596908', 'NCT04373317', 'NCT01892176', 'NCT01016743', 'NCT01399905', 'NCT00359723', 'NCT00758368', 'NCT03665454', 'NCT02565628', 'NCT03446807', 'NCT04858074']
Query 75 top bi-encoder score: 1.7491
Query 75: Top bi-encoder score 1.7491 exceeds threshold 1.6, skipping cross-encoder re-ranking
  Query 75 stage 2 retrieved (final): ['NCT00443768', 'NCT00001928', 'NCT00203060', 'NCT00199394', 'NCT04380142', 'NCT00526630', 'NCT00607451', 'NCT00148512', 'NCT00472355', 'NCT00666653', 'NCT00307450', 'NCT03956979', 'NCT00199407',

### Bi-encoder and cross-encoder on selected documents based on Score Range threshold

In [ ]:
"""
Zuveir Jameer
2 April 2025
Bi-encoder and cross-encoder selective based on score range threshold
"""

import json
import numpy as np
from elasticsearch import Elasticsearch
from tqdm import tqdm
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer
import time
from datetime import datetime
import pandas as pd

# Cross-encoder for re-ranking

cross_tokenizer = AutoTokenizer.from_pretrained("cross-encoder/ms-marco-MiniLM-L6-v2")
cross_model = AutoModelForSequenceClassification.from_pretrained("cross-encoder/ms-marco-MiniLM-L6-v2")

cross_model.eval()
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
cross_model.to(device)

# Evaluation Metric Functions
def precision_at_k(re_ranked_docs, relevant_docs, k):
    top_k_docs = re_ranked_docs[:k]
    num_relevant_in_k = sum(1 for doc in top_k_docs 
                            if doc in relevant_docs and relevant_docs[doc] > 0)
    return num_relevant_in_k / k if k else 0.0

def recall_at_k(re_ranked_docs, relevant_docs, k):
    top_k_docs = re_ranked_docs[:k]
    total_relevant = sum(1 for doc, rel in relevant_docs.items() if rel > 0)
    if total_relevant == 0:
        return 0.0
    num_relevant_in_k = sum(1 for doc in top_k_docs 
                            if doc in relevant_docs and relevant_docs[doc] > 0)
    return num_relevant_in_k / total_relevant

def f1_at_k(re_ranked_docs, relevant_docs, k):
    p = precision_at_k(re_ranked_docs, relevant_docs, k)
    r = recall_at_k(re_ranked_docs, relevant_docs, k)
    if (p + r) == 0:
        return 0.0
    return 2 * (p * r) / (p + r)

def average_precision(re_ranked_docs, relevant_docs):
    if not relevant_docs:
        return 0.0
    
    relevant_indices = [
        i for i, doc in enumerate(re_ranked_docs)
        if doc in relevant_docs and relevant_docs[doc] > 0
    ]
    if not relevant_indices:
        return 0.0
    
    ap_sum = 0.0
    for idx, rank_pos in enumerate(relevant_indices, start=1):
        prec = precision_at_k(re_ranked_docs, relevant_docs, rank_pos+1)
        ap_sum += prec
    return ap_sum / len(relevant_indices)

def ndcg_at_k(re_ranked_docs, relevant_docs, k):
    def dcg(scores):
        return sum(((2**s - 1) / np.log2(idx + 2)) for idx, s in enumerate(scores))
    
    top_k_docs = re_ranked_docs[:k]
    actual_scores = [relevant_docs[doc] if doc in relevant_docs else 0 for doc in top_k_docs]
    actual_dcg = dcg(actual_scores)
    
    sorted_rels = sorted(relevant_docs.values(), reverse=True)[:k]
    ideal_dcg = dcg(sorted_rels)
    
    if ideal_dcg == 0:
        return 0.0
    return actual_dcg / ideal_dcg

def mrr_at_k(re_ranked_docs, relevant_docs, k):
    for i, doc_id in enumerate(re_ranked_docs[:k], start=1):
        if doc_id in relevant_docs and relevant_docs[doc_id] > 0:
            return 1.0 / i
    return 0.0

# Data Loading
def load_jsonl(filename):
    with open(filename, "r") as f:
        return [json.loads(line) for line in f]

queries = load_jsonl("trec_ct_2021_query.jsonl")
qrels_list = load_jsonl("trec_ct_2021_qrels.jsonl")

# Create a dictionary mapping query_id to {doc_id: relevance}
qrels = {}
for q in qrels_list:
    qid = q["query_id"]
    if qid not in qrels:
        qrels[qid] = {}
    relevance = int(q.get("relevance", 0))
    qrels[qid][q["doc_id"]] = relevance

print(f"Loaded {len(queries)} queries")
print(f"Loaded qrels for {len(qrels)} queries")

RELEVANCE_THRESHOLD = 2

filtered_qrels = {
    qid: {doc: rel for doc, rel in rels.items() if rel >= RELEVANCE_THRESHOLD}
    for qid, rels in qrels.items()
}

ndcg_qrels = {
    qid: {doc: rel for doc, rel in rels.items()}
    for qid, rels in qrels.items()
}


# Dense Retrieval (Stage 1) Functions using Bi-Encoder

def search_detailed_descrption(query_vector, K=100):
    """
    Returns a list of tuples: (doc_id, similarity_score)
    """
    search_body = {
      "query": {
        "script_score": {
          "query": {
            "bool": {
              "must": [
                {"match_all": {}},
                {"exists": {"field": "detailed_descrption_vector_1"}}
              ]
            }
          },
          "script": {
            "source": "cosineSimilarity(params.query_vector, 'detailed_descrption_vector_1') + 1.0",
            "params": {"query_vector": query_vector}
          }
        }
      }
    }
    response = es.search(index="ir_dev_sentence_index", body=search_body, size=K)
    results = []
    for hit in response['hits']['hits']:
        doc_id = hit["_source"]["doc_id"]
        score = hit["_score"]
        results.append((doc_id, score))
    return results

def get_document_text(doc_id):
    """
    Retrieve the detailed description for a document given its ID.
    """
    query = {
        "query": {
            "term": {"doc_id.keyword": doc_id}
        }
    }
    response = es.search(index="ir_dev_sentence_index", body=query, size=1)
    hits = response.get("hits", {}).get("hits", [])
    if hits:
        doc_text = hits[0]["_source"].get("detailed_descrption", "").strip()
        return doc_text if doc_text else None
    else:
        return None

K = 20  # Number of candidates from dense retrieval
top_k = 10  # Number of results from re-ranker

def cross_encoder_rerank(query_text, candidate_doc_ids, top_k=top_k):
    candidate_pairs = []
    valid_doc_ids = []
    for doc_id in candidate_doc_ids:
        try:
            doc_text = get_document_text(doc_id)
            if doc_text is None or not doc_text.strip():
                continue
            candidate_pairs.append((query_text, doc_text))
            valid_doc_ids.append(doc_id)
        except Exception as e:
            print(f"Error fetching doc {doc_id}: {e}")
            continue

    if not candidate_pairs:
        print("No valid candidate pairs found; returning original candidates.")
        return candidate_doc_ids

    inputs = cross_tokenizer.batch_encode_plus(
        candidate_pairs, 
        padding=True, 
        truncation=True, 
        return_tensors="pt"
    )
    inputs = {k: v.to(cross_model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = cross_model(**inputs)

    if outputs.logits.shape[-1] == 1:
        scores = outputs.logits.squeeze(-1)
    else:
        scores = torch.softmax(outputs.logits, dim=1)[:, 1]

    ranked = sorted(
        zip(valid_doc_ids, scores.tolist()), 
        key=lambda x: x[1], 
        reverse=True
    )

    if top_k is not None:
        ranked = ranked[:top_k]

    return [doc_id for doc_id, score in ranked]


# Main Processing Loop
all_metrics = []

# Define a threshold on the range (max - min) of the bi-encoder scores.
# A small range indicates that the model is not discriminating well.
score_range_threshold = 0.1

print("Processing queries and retrieving documents...")
for query in tqdm(queries, total=len(queries)):
    qid = query["query_id"]
    query_text = query["text"]
    if not query_text.strip():
        print(f"Query {qid} is empty, skipping.")
        continue

    start_time = time.time()

    # Stage 1: Get the dense vector for the query and retrieve candidates
    query_vector = get_embedding(query_text) 
    try:
        # retrieved_docs_with_scores is a list of tuples: (doc_id, score)
        retrieved_docs_with_scores = search_detailed_descrption(query_vector, K)
        if not retrieved_docs_with_scores:
            print(f"No documents retrieved for query {qid}")
            continue
        # Separate doc_ids and scores
        retrieved_docs = [doc for doc, score in retrieved_docs_with_scores]
        scores = [score for doc, score in retrieved_docs_with_scores]
        top_score = max(scores)
        score_range = max(scores) - min(scores)
        print(f"Query {qid} Stage 1 retrieved docs: {retrieved_docs}")
        print(f"Query {qid} top bi-encoder score: {top_score:.4f}")
        print(f"Query {qid} score range: {score_range:.4f}")
    except Exception as e:
        print(f"Error retrieving documents for query {qid}: {e}")
        continue

    # Stage 2: Decide whether to apply cross-encoder based on the score distribution.
    if score_range < score_range_threshold:
        print(f"Query {qid}: Score range {score_range:.4f} is below threshold {score_range_threshold}, applying cross-encoder re-ranking")
        try:
            re_ranked_docs = cross_encoder_rerank(query_text, retrieved_docs)
        except Exception as e:
            print(f"Error in cross encoder re-ranking for query {qid}: {e}")
            re_ranked_docs = retrieved_docs
    else:
        print(f"Query {qid}: Score range {score_range:.4f} exceeds threshold {score_range_threshold}, skipping cross-encoder re-ranking")
        re_ranked_docs = retrieved_docs

    total_time = time.time() - start_time

    # Evaluate using filtered qrels (ensure your evaluation functions are correct)
    relevant = filtered_qrels.get(qid, {})
    ndcg_relevant = ndcg_qrels.get(qid, {})
    p10 = precision_at_k(re_ranked_docs, relevant, top_k)
    r10 = recall_at_k(re_ranked_docs, relevant, top_k)
    f1 = f1_at_k(re_ranked_docs, relevant, top_k)
    ap = average_precision(re_ranked_docs, relevant)
    ndcg = ndcg_at_k(re_ranked_docs, ndcg_relevant, top_k)
    mrr_val = mrr_at_k(re_ranked_docs, relevant, top_k)

    all_metrics.append({
        "query_id": qid, 
        "precision": p10, 
        "recall": r10, 
        "f1": f1, 
        "ap": ap, 
        "ndcg": ndcg,
        "mrr": mrr_val,
        "time_seconds": total_time
    })
    print(f"  Query {qid} final retrieved docs: {re_ranked_docs}")
    print(f"   Query {qid} metrics: Precision@{top_k}: {p10:.4f}, Recall@{top_k}: {r10:.4f}, F1@{top_k}: {f1:.4f}, AP: {ap:.4f}, nDCG@{top_k}: {ndcg:.4f}, MRR@{top_k}: {mrr_val:.4f}")
    print(f"    Total time for query: {total_time:.2f} seconds")

if all_metrics:
    avg_precision = np.mean([m["precision"] for m in all_metrics])
    avg_recall = np.mean([m["recall"] for m in all_metrics])
    avg_f1 = np.mean([m["f1"] for m in all_metrics])
    avg_ap = np.mean([m["ap"] for m in all_metrics])
    avg_ndcg = np.mean([m["ndcg"] for m in all_metrics])
    avg_mrr = np.mean([m["mrr"] for m in all_metrics])
    avg_time = np.mean([m["time_seconds"] for m in all_metrics])

    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"Date/time:{current_time}")
    print("\nBi-encoder and cross-encoder - Overall evaluation metrics:")
    print(f"Relevance Threshold: {RELEVANCE_THRESHOLD}")
    print("Mean Precision@{}: {:.4f}".format(top_k, avg_precision))
    print("Mean Recall@{}: {:.4f}".format(top_k, avg_recall))
    print("Mean F1@{}: {:.4f}".format(top_k, avg_f1))
    print("Mean Average Precision (MAP): {:.4f}".format(avg_ap))
    print("Mean nDCG@{}: {:.4f}".format(top_k, avg_ndcg))
    print(f"Mean MRR@{top_k}: {avg_mrr:.4f}")
    print(f"Average total time per query: {avg_time:.2f} seconds")
else:
    print("No query metrics to average.")


# Write detailed query metrics to an Excel file.
query_metrics_df = pd.DataFrame(all_metrics)
query_metrics_df = query_metrics_df[[
    "query_id",
    "precision",
    "recall",
    "f1",
    "ap",
    "ndcg",
    "mrr",
    "time_seconds"
]]
query_metrics_file = f"sentence_embedding_bi_and_Cross-encoding_metrics_score_range_threshold_{score_range_threshold}.xlsx"
query_metrics_df.to_excel(query_metrics_file, index=False)
print(f"Metrics successfully exported to {query_metrics_file}")

# Write aggregated results to another Excel file.
aggregated_data = {
    "Test": ["Bi-encoder + Cross-encoder score range"],
    "Date/Time": [current_time],
    "Relevance Threshold": [RELEVANCE_THRESHOLD],
    f"Mean Precision@{top_k}": [avg_precision],
    f"Mean Recall@{top_k}": [avg_recall],
    f"Mean F1@{top_k}": [avg_f1],
    "Mean Average Precision (MAP)": [avg_ap],
    f"Mean nDCG@{top_k}": [avg_ndcg],
    f"Mean MRR@{top_k}": [avg_mrr],
    "Avg Time per Query (s)": [avg_time],
}
aggregated_df = pd.DataFrame(aggregated_data)
aggregated_file = f"evaluation_results_sentence_embeddings_bi_and_Cross-encoding_score_range_threshold_{score_range_threshold}.xlsx"
aggregated_df.to_excel(aggregated_file, index=False)
print(f"Aggregated metrics successfully exported to {aggregated_file}")

Loaded 75 queries
Loaded qrels for 75 queries
Processing queries and retrieving documents...


  0%|          | 0/75 [00:00<?, ?it/s]/var/folders/b3/59dkzgnj0zxdhdx31d2ztm000000gn/T/ipykernel_54942/3806370774.py:145: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.search(index="ir_dev_sentence_index", body=search_body, size=K)
  1%|▏         | 1/75 [00:00<00:17,  4.27it/s]

Query 1 Stage 1 retrieved docs: ['NCT00003470', 'NCT01502241', 'NCT00003537', 'NCT00003621', 'NCT00003471', 'NCT00004259', 'NCT00783393', 'NCT00003775', 'NCT03906448', 'NCT00033280', 'NCT02755987', 'NCT00028795', 'NCT03633552', 'NCT00841555', 'NCT00003375', 'NCT00473408', 'NCT00002569', 'NCT00003465', 'NCT00360828', 'NCT03022578']
Query 1 top bi-encoder score: 1.7383
Query 1 score range: 0.1176
Query 1: Score range 0.1176 exceeds threshold 0.1, skipping cross-encoder re-ranking
  Query 1 final retrieved docs: ['NCT00003470', 'NCT01502241', 'NCT00003537', 'NCT00003621', 'NCT00003471', 'NCT00004259', 'NCT00783393', 'NCT00003775', 'NCT03906448', 'NCT00033280', 'NCT02755987', 'NCT00028795', 'NCT03633552', 'NCT00841555', 'NCT00003375', 'NCT00473408', 'NCT00002569', 'NCT00003465', 'NCT00360828', 'NCT03022578']
   Query 1 metrics: Precision@10: 0.4000, Recall@10: 0.0851, F1@10: 0.1404, AP: 0.8042, nDCG@10: 0.6502, MRR@10: 1.0000
    Total time for query: 0.23 seconds


  3%|▎         | 2/75 [00:00<00:16,  4.40it/s]

Query 2 Stage 1 retrieved docs: ['NCT03628313', 'NCT02753023', 'NCT02395107', 'NCT03332745', 'NCT01157572', 'NCT02010177', 'NCT04061213', 'NCT03863132', 'NCT02792452', 'NCT00590135', 'NCT02531581', 'NCT04140019', 'NCT01980797', 'NCT01575249', 'NCT00624884', 'NCT02316587', 'NCT04344353', 'NCT02344199', 'NCT03867708', 'NCT04144725']
Query 2 top bi-encoder score: 1.7217
Query 2 score range: 0.1293
Query 2: Score range 0.1293 exceeds threshold 0.1, skipping cross-encoder re-ranking
  Query 2 final retrieved docs: ['NCT03628313', 'NCT02753023', 'NCT02395107', 'NCT03332745', 'NCT01157572', 'NCT02010177', 'NCT04061213', 'NCT03863132', 'NCT02792452', 'NCT00590135', 'NCT02531581', 'NCT04140019', 'NCT01980797', 'NCT01575249', 'NCT00624884', 'NCT02316587', 'NCT04344353', 'NCT02344199', 'NCT03867708', 'NCT04144725']
   Query 2 metrics: Precision@10: 0.3000, Recall@10: 0.0217, F1@10: 0.0405, AP: 0.4955, nDCG@10: 0.5271, MRR@10: 1.0000
    Total time for query: 0.22 seconds


...

  Query 73 final retrieved docs: ['NCT04215601', 'NCT02774434', 'NCT02446951', 'NCT01434810', 'NCT01622699', 'NCT03942757', 'NCT04099602', 'NCT02625688', 'NCT01136577', 'NCT03183986']
   Query 73 metrics: Precision@10: 0.4000, Recall@10: 0.2000, F1@10: 0.2667, AP: 0.4500, nDCG@10: 0.5321, MRR@10: 0.5000
    Total time for query: 0.82 seconds


 99%|█████████▊| 74/75 [00:48<00:00,  1.92it/s]

Query 74 Stage 1 retrieved docs: ['NCT00306046', 'NCT00051636', 'NCT00774020', 'NCT00480662', 'NCT00747994', 'NCT04222452', 'NCT02106455', 'NCT00894075', 'NCT02802384', 'NCT02056730', 'NCT00453128', 'NCT03993821', 'NCT03859895', 'NCT00108394', 'NCT00195936', 'NCT00066183', 'NCT00417612', 'NCT02198001', 'NCT02967042', 'NCT00973336']
Query 74 top bi-encoder score: 1.8162
Query 74 score range: 0.2708
Query 74: Score range 0.2708 exceeds threshold 0.1, skipping cross-encoder re-ranking
  Query 74 final retrieved docs: ['NCT00306046', 'NCT00051636', 'NCT00774020', 'NCT00480662', 'NCT00747994', 'NCT04222452', 'NCT02106455', 'NCT00894075', 'NCT02802384', 'NCT02056730', 'NCT00453128', 'NCT03993821', 'NCT03859895', 'NCT00108394', 'NCT00195936', 'NCT00066183', 'NCT00417612', 'NCT02198001', 'NCT02967042', 'NCT00973336']
   Query 74 metrics: Precision@10: 0.7000, Recall@10: 0.6364, F1@10: 0.6667, AP: 0.9478, nDCG@10: 0.7885, MRR@10: 1.0000
    Total time for query: 0.25 seconds
Query 75 Stage 1 re

100%|██████████| 75/75 [00:49<00:00,  1.52it/s]

  Query 75 final retrieved docs: ['NCT00001928', 'NCT03568968', 'NCT00443768', 'NCT00607451', 'NCT00199394', 'NCT00307450', 'NCT00472355', 'NCT04472130', 'NCT03433950', 'NCT00199420']
   Query 75 metrics: Precision@10: 0.3000, Recall@10: 0.0213, F1@10: 0.0397, AP: 0.7917, nDCG@10: 0.5685, MRR@10: 1.0000
    Total time for query: 0.84 seconds
Date/time:2025-04-10 08:26:29

Bi-encoder and cross-encoder - Overall evaluation metrics:
Relevance Threshold: 2
Mean Precision@10: 0.3147
Mean Recall@10: 0.0725
Mean F1@10: 0.1035
Mean Average Precision (MAP): 0.4531
Mean nDCG@10: 0.4215
Mean MRR@10: 0.5234
Average total time per query: 0.66 seconds
